## Import Libraries

In [1]:
import pandas as pd
import numpy as np

# **Import scrapped CSV**

In [2]:
df_fight_stats =pd.read_csv('scrape_ufc_stats-main/ufc_fight_stats.csv')
df_event_details =pd.read_csv('scrape_ufc_stats-main/ufc_event_details.csv')
df_fighter_tott = pd.read_csv('scrape_ufc_stats-main/ufc_fighter_tott.csv')
df_results = pd.read_csv('scrape_ufc_stats-main/ufc_fight_results.csv')
df_fighter_details =pd.read_csv('scrape_ufc_stats-main/ufc_fighter_details.csv')

In [3]:
import re
def extract_rounds(text):
  if text == "No Time Limit":
    return [0]
  else:
    result = re.findall(r'\((.*?)\)', text)
    result = list(map(int,result[0].split("-")))
    return result

def handle_result_df(df):
  WEIGHTCLASSES_my = ['Lightweight', 'Welterweight', 'Middleweight', 'Featherweight','Heavyweigh', 'Light Heavyweight', 'Bantamweight', 'Flyweight', 'Strawweight', 'Open Weight', 'Catch Weight']
  df['Winner'] = df.apply(lambda row: 'NC' if row['OUTCOME'] == 'NC/NC' else 'DRAW' if row['OUTCOME'] == 'D/D' else row['BOUT'].split(' vs. ')[(row['OUTCOME'].split('/')).index('W')], axis=1)
  WeightClasses_df = df['WEIGHTCLASS']
  result=[]
  women = []
  Title = []
  for bout in WeightClasses_df:
    bout_low = bout.lower()
    women.append(1) if 'women' in bout_low else women.append(0)
    Title.append(1) if 'title' in bout_low else Title.append(0)
    find = False
    for classes in WEIGHTCLASSES_my:   
      if classes.lower() in bout_low.lower():
        result.append(classes)
        find = True      
        break
    if not find:
      result.append('Open Weight')
  
  df.loc[:,"WeightClass"] = result
  df.loc[:,"Tittle"] = Title
  df.loc[:,"Women_fight"] = women
  df['TimeSec'] = df['TIME'].apply(lambda x: (datetime.strptime(x, '%M:%S') - datetime(1900,1,1)).total_seconds()).astype(int)
  df['TIME FORMAT'] = df['TIME FORMAT'].apply(lambda x: extract_rounds(str(x)))
  df_results['TotalTime'] = df_results.apply(lambda row: sum(row['TIME FORMAT'][0:int(row['ROUND'])-1])*60 + row['TimeSec'],axis=1)
  df = df.drop(['OUTCOME','WEIGHTCLASS','TimeSec'],axis=1)
  df_results = df_results.merge(df_event_details[['EVENT','DATE']],how='left', left_on='EVENT', right_on='EVENT')
  df = df[['DATE','EVENT', 'BOUT', 'Winner', 'WeightClass', 'Tittle', 'Women_fight','METHOD','TIME FORMAT', 'ROUND', 'TIME', 'TotalTime', 'REFEREE',
       'DETAILS', 'URL']]
  df['BOUT'] = df['BOUT'].str.split().str.join(' ')
  df['EVENT'] = df['EVENT'].str.split().str.join(' ')
  return df

# **handle duplicated fighters name**

In [4]:
df_fighter_tott.shape

(3923, 7)

check each fighter if ha has fight in UFC (DWCS not included) if not drop fighters - not solved duplicated values

In [5]:
df_fighter_tott['Has_fight'] = df_fighter_tott['FIGHTER'].apply(lambda x: x in list(df_fight_stats['FIGHTER']))

In [6]:
df_fighter_tott[df_fighter_tott['Has_fight'] == False]

,FIGHTER,HEIGHT,WEIGHT,REACH,STANCE,DOB,URL,Has_fight
0,Tom Aaron,--,155 lbs.,--,NaN,"Jul 13, 1978",http://ufcstats.com/fighter-details/93fe7332d1...,False
4,Hiroyuki Abe,"5' 6""",145 lbs.,--,Orthodox,--,http://ufcstats.com/fighter-details/c0ed7b2081...,False
9,Daniel Acacio,"5' 8""",180 lbs.,--,Orthodox,"Dec 27, 1977",http://ufcstats.com/fighter-details/1c5879330d...,False
12,Anthony Adams,"6' 1""",185 lbs.,"76""",Orthodox,"Jan 13, 1988",http://ufcstats.com/fighter-details/a77633a989...,False
19,Fabio Aguiar,"6' 0""",185 lbs.,--,NaN,"Feb 10, 1988",http://ufcstats.com/fighter-details/501821d7fb...,False
...,...,...,...,...,...,...,...,...
3918,Luan Lacerda,"5' 7""",135 lbs.,--,NaN,"Jan 07, 1993",http://ufcstats.com/fighter-details/6fc506e109...,False
3919,Nick Fiore,"5' 11""",155 lbs.,--,NaN,"Dec 10, 1997",http://ufcstats.com/fighter-details/036e96c1c1...,False
3920,Shara Magomedov,"6' 2""",170 lbs.,--,NaN,"May 16, 1994",http://ufcstats.com/fighter-details/06734ca9d8...,False
3921,Melquizael Costa,"5' 10""",155 lbs.,--,NaN,"Sep 14, 1996",http://ufcstats.com/fighter-details/20bccc9bb4...,False


In [7]:
df_fighter_tott = df_fighter_tott.loc[df_fighter_tott['Has_fight']==True,:]

In [8]:
df_fighter_tott

,FIGHTER,HEIGHT,WEIGHT,REACH,STANCE,DOB,URL,Has_fight
1,Danny Abbadi,"5' 11""",155 lbs.,--,Orthodox,"Jul 03, 1983",http://ufcstats.com/fighter-details/15df64c02b...,True
2,David Abbott,"6' 0""",265 lbs.,--,Switch,--,http://ufcstats.com/fighter-details/b361180739...,True
3,Shamil Abdurakhimov,"6' 3""",235 lbs.,"76""",Orthodox,"Sep 02, 1981",http://ufcstats.com/fighter-details/2f5cbecbbe...,True
5,Daichi Abe,"5' 11""",170 lbs.,"71""",Orthodox,"Nov 27, 1991",http://ufcstats.com/fighter-details/5140122c3e...,True
6,Papy Abedi,"5' 11""",185 lbs.,--,Southpaw,"Jun 30, 1978",http://ufcstats.com/fighter-details/c9f6385af6...,True
...,...,...,...,...,...,...,...,...
3901,William Gomis,--,145 lbs.,--,NaN,"Jun 13, 1997",http://ufcstats.com/fighter-details/e1d40e8782...,True
3905,Don Shainis,"5' 6""",145 lbs.,--,NaN,"Sep 16, 1990",http://ufcstats.com/fighter-details/da4628f173...,True
3909,Lucas Alexander,"5' 10""",145 lbs.,--,NaN,"Jul 28, 1995",http://ufcstats.com/fighter-details/11583163f4...,True
3912,Carlos Mota,"5' 6""",125 lbs.,--,NaN,"Apr 15, 1995",http://ufcstats.com/fighter-details/b15862fcd6...,True


In [9]:
df_fighter_tott['WEIGHT'] = pd.to_numeric(df_fighter_tott['WEIGHT'].str.replace('lbs.','',regex=True), errors='coerce')
df_fighter_tott['REACH'] = pd.to_numeric(df_fighter_tott['REACH'].str.replace('"','',regex=True), errors='coerce')

In [10]:
df_fighter_tott['HEIGHT'] = df_fighter_tott['HEIGHT'].replace('--',np.nan)
df_fighter_tott[['ft','inch']] = df_fighter_tott['HEIGHT'].str.extractall(r"(\d+)").unstack().astype(float)
df_fighter_tott['HEIGHT'] = df_fighter_tott['ft']*12 + df_fighter_tott['inch']

In [11]:
df_fighter_tott = df_fighter_tott.drop(['ft', 'inch'], axis =1)

**drop fighters dupplicated fighters which has no start in ufc**

In [12]:
df_fighter_tott.drop(df_fighter_tott[df_fighter_tott['URL']=='http://ufcstats.com/fighter-details/c8661e204c66f325'].index, inplace=True)
df_fighter_tott.drop(df_fighter_tott[df_fighter_tott['URL']=='http://ufcstats.com/fighter-details/3a28e1e641366308'].index, inplace=True)
df_fighter_tott.drop(df_fighter_tott[df_fighter_tott['URL']=='http://ufcstats.com/fighter-details/d52ef694108f8235'].index, inplace=True)

find fighters with same names and put nickname to one of them

In [13]:
duplicated_fighters_index = list(df_fighter_tott[df_fighter_tott['FIGHTER'].duplicated()].index)[0]


In [14]:
duplicated_fighters_index

3109

In [15]:
df_fighter_tott[df_fighter_tott['FIGHTER'] == df_fighter_tott.loc[duplicated_fighters_index,'FIGHTER']]

,FIGHTER,HEIGHT,WEIGHT,REACH,STANCE,DOB,URL,Has_fight
3108,Bruno Silva,64.0,125.0,65.0,Orthodox,"Mar 16, 1990",http://ufcstats.com/fighter-details/294aa73dbf...,True
3109,Bruno Silva,72.0,185.0,74.0,Orthodox,"Jul 13, 1989",http://ufcstats.com/fighter-details/12ebd7d157...,True


In [16]:
name_dup_fighter = df_fighter_tott.loc[duplicated_fighters_index,'FIGHTER']
url_dup_fighter = df_fighter_tott.loc[duplicated_fighters_index,'URL']
url_dup_fighter

'http://ufcstats.com/fighter-details/12ebd7d157e91701'

In [17]:
df_fighter_details.head()

,FIRST,LAST,NICKNAME,URL
0,Tom,Aaron,NaN,http://ufcstats.com/fighter-details/93fe7332d1...
1,Danny,Abbadi,The Assassin,http://ufcstats.com/fighter-details/15df64c02b...
2,Nariman,Abbasov,Bayraktar,http://ufcstats.com/fighter-details/59a9d6dac6...
3,David,Abbott,Tank,http://ufcstats.com/fighter-details/b361180739...
4,Hamdy,Abdelwahab,The Hammer,http://ufcstats.com/fighter-details/3329d692ae...


In [18]:
nickname=df_fighter_details.loc[df_fighter_details['URL']==url_dup_fighter, 'NICKNAME'].values[0]
weight_class ='Middleweight'
nickname

'Blindado'

**update name in fighter tott df**

In [19]:
df_fighter_tott.loc[df_fighter_tott['URL'] == url_dup_fighter, 'FIGHTER'] = df_fighter_tott.loc[df_fighter_tott['URL'] == url_dup_fighter, 'FIGHTER'] + ' ' + nickname

In [20]:
df_fighter_tott.loc[df_fighter_tott['URL'] == url_dup_fighter, 'FIGHTER']

3109    Bruno Silva Blindado
Name: FIGHTER, dtype: object

In [21]:
fights_list = df_results[(df_results['BOUT'].str.contains(name_dup_fighter)) & (df_results['WEIGHTCLASS'].str.contains(weight_class))]['BOUT'].str.replace(" ", "").tolist()
fights_list

['BrunoSilvavs.GeraldMeerschaert',
 'AlexPereiravs.BrunoSilva',
 'JordanWrightvs.BrunoSilva',
 'AndrewSanchezvs.BrunoSilva',
 'WellingtonTurmanvs.BrunoSilva']

**update name in fight stats df**

In [22]:
df_fight_stats.loc[(df_fight_stats['BOUT'].str.replace(" ", "").isin(fights_list)) & (df_fight_stats['FIGHTER']==name_dup_fighter),'FIGHTER'] = name_dup_fighter + ' ' + nickname

In [23]:
df_fight_stats.loc[(df_fight_stats['BOUT'].str.replace(" ", "").isin(fights_list))]

,EVENT,BOUT,ROUND,FIGHTER,KD,SIG.STR.,SIG.STR. %,TOTAL STR.,TD,TD %,SUB.ATT,REV.,CTRL,HEAD,BODY,LEG,DISTANCE,CLINCH,GROUND
784,UFC Fight Night: Vera vs. Cruz,Bruno Silva vs. Gerald Meerschaert,Round 1,Bruno Silva Blindado,0.0,10 of 26,38%,19 of 38,0 of 0,---,0.0,0.0,0:00,5 of 17,3 of 6,2 of 3,9 of 25,1 of 1,0 of 0
785,UFC Fight Night: Vera vs. Cruz,Bruno Silva vs. Gerald Meerschaert,Round 2,Bruno Silva Blindado,0.0,12 of 41,29%,12 of 41,0 of 0,---,0.0,0.0,0:00,8 of 32,2 of 7,2 of 2,12 of 41,0 of 0,0 of 0
786,UFC Fight Night: Vera vs. Cruz,Bruno Silva vs. Gerald Meerschaert,Round 3,Bruno Silva Blindado,0.0,7 of 14,50%,7 of 14,0 of 0,---,0.0,0.0,0:01,4 of 11,1 of 1,2 of 2,6 of 13,1 of 1,0 of 0
787,UFC Fight Night: Vera vs. Cruz,Bruno Silva vs. Gerald Meerschaert,Round 1,Gerald Meerschaert,0.0,12 of 38,31%,20 of 52,1 of 1,100%,0.0,0.0,1:55,8 of 33,2 of 2,2 of 3,6 of 25,0 of 0,6 of 13
788,UFC Fight Night: Vera vs. Cruz,Bruno Silva vs. Gerald Meerschaert,Round 2,Gerald Meerschaert,0.0,19 of 51,37%,19 of 51,0 of 2,0%,0.0,0.0,0:03,8 of 38,6 of 8,5 of 5,19 of 50,0 of 0,0 of 1
789,UFC Fight Night: Vera vs. Cruz,Bruno Silva vs. Gerald Meerschaert,Round 3,Gerald Meerschaert,1.0,15 of 25,60%,22 of 33,0 of 3,0%,1.0,0.0,0:14,12 of 21,1 of 2,2 of 2,11 of 20,0 of 0,4 of 5
1894,UFC Fight Night: Santos vs. Ankalaev,Alex Pereira vs. Bruno Silva,Round 1,Alex Pereira,0.0,27 of 51,52%,31 of 55,0 of 0,---,0.0,0.0,0:00,10 of 28,11 of 16,6 of 7,23 of 45,4 of 6,0 of 0
1895,UFC Fight Night: Santos vs. Ankalaev,Alex Pereira vs. Bruno Silva,Round 2,Alex Pereira,0.0,33 of 58,56%,53 of 79,0 of 0,---,0.0,0.0,0:07,19 of 41,13 of 16,1 of 1,30 of 53,3 of 4,0 of 1
1896,UFC Fight Night: Santos vs. Ankalaev,Alex Pereira vs. Bruno Silva,Round 3,Alex Pereira,0.0,48 of 70,68%,81 of 105,0 of 0,---,0.0,0.0,1:10,31 of 51,17 of 18,0 of 1,33 of 53,15 of 17,0 of 0
1897,UFC Fight Night: Santos vs. Ankalaev,Alex Pereira vs. Bruno Silva,Round 1,Bruno Silva Blindado,0.0,16 of 46,34%,19 of 49,1 of 1,100%,0.0,0.0,0:48,10 of 40,2 of 2,4 of 4,13 of 42,3 of 4,0 of 0


# **Fight_stats_details df**

In [24]:
df_fight_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32212 entries, 0 to 32211
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   EVENT       32212 non-null  object 
 1   BOUT        32212 non-null  object 
 2   ROUND       32170 non-null  object 
 3   FIGHTER     32170 non-null  object 
 4   KD          32170 non-null  float64
 5   SIG.STR.    32170 non-null  object 
 6   SIG.STR. %  32170 non-null  object 
 7   TOTAL STR.  32170 non-null  object 
 8   TD          32170 non-null  object 
 9   TD %        32170 non-null  object 
 10  SUB.ATT     32170 non-null  float64
 11  REV.        32170 non-null  float64
 12  CTRL        32170 non-null  object 
 13  HEAD        32170 non-null  object 
 14  BODY        32170 non-null  object 
 15  LEG         32170 non-null  object 
 16  DISTANCE    32170 non-null  object 
 17  CLINCH      32170 non-null  object 
 18  GROUND      32170 non-null  object 
dtypes: float64(3), object(16)

In [25]:
df_fight_stats.head()

,EVENT,BOUT,ROUND,FIGHTER,KD,SIG.STR.,SIG.STR. %,TOTAL STR.,TD,TD %,SUB.ATT,REV.,CTRL,HEAD,BODY,LEG,DISTANCE,CLINCH,GROUND
0,UFC Fight Night: Cannonier vs. Strickland,Jared Cannonier vs. Sean Strickland,Round 1,Jared Cannonier,0.0,16 of 43,37%,16 of 43,0 of 0,---,0.0,0.0,0:00,3 of 26,7 of 8,6 of 9,16 of 43,0 of 0,0 of 0
1,UFC Fight Night: Cannonier vs. Strickland,Jared Cannonier vs. Sean Strickland,Round 2,Jared Cannonier,0.0,25 of 58,43%,25 of 58,0 of 0,---,0.0,0.0,0:00,6 of 28,11 of 18,8 of 12,25 of 58,0 of 0,0 of 0
2,UFC Fight Night: Cannonier vs. Strickland,Jared Cannonier vs. Sean Strickland,Round 3,Jared Cannonier,0.0,33 of 69,47%,33 of 69,0 of 0,---,0.0,0.0,0:00,12 of 40,15 of 21,6 of 8,33 of 69,0 of 0,0 of 0
3,UFC Fight Night: Cannonier vs. Strickland,Jared Cannonier vs. Sean Strickland,Round 4,Jared Cannonier,0.0,27 of 61,44%,27 of 61,0 of 0,---,0.0,0.0,0:00,14 of 39,11 of 18,2 of 4,27 of 61,0 of 0,0 of 0
4,UFC Fight Night: Cannonier vs. Strickland,Jared Cannonier vs. Sean Strickland,Round 5,Jared Cannonier,0.0,40 of 79,50%,40 of 79,0 of 0,---,0.0,0.0,0:00,22 of 54,16 of 23,2 of 2,40 of 79,0 of 0,0 of 0


**drop rows where are not fighter filled**

In [26]:
df_fight_stats = df_fight_stats[df_fight_stats['FIGHTER'].notna()]

**splits stats by 'of' to attemp, landed**

In [27]:
cols_to_split = [col for col in list(df_fight_stats.columns) if ' of ' in str(df_fight_stats.loc[0,col])]

In [28]:
for col in cols_to_split:
  print(col + '_landed')
  df_fight_stats[[col + '_landed', col + '_attemps']] = (df_fight_stats[col].str.split(' of ', expand=True))
  df_fight_stats[[col + '_landed', col + '_attemps']] = df_fight_stats[[col + '_landed', col + '_attemps']].fillna(0)
  df_fight_stats[[col + '_landed', col + '_attemps']] = df_fight_stats[[col + '_landed', col + '_attemps']].astype(int)

SIG.STR._landed
TOTAL STR._landed
TD_landed
HEAD_landed
BODY_landed
LEG_landed
DISTANCE_landed
CLINCH_landed
GROUND_landed


In [29]:
df_fight_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32170 entries, 0 to 32211
Data columns (total 37 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   EVENT               32170 non-null  object 
 1   BOUT                32170 non-null  object 
 2   ROUND               32170 non-null  object 
 3   FIGHTER             32170 non-null  object 
 4   KD                  32170 non-null  float64
 5   SIG.STR.            32170 non-null  object 
 6   SIG.STR. %          32170 non-null  object 
 7   TOTAL STR.          32170 non-null  object 
 8   TD                  32170 non-null  object 
 9   TD %                32170 non-null  object 
 10  SUB.ATT             32170 non-null  float64
 11  REV.                32170 non-null  float64
 12  CTRL                32170 non-null  object 
 13  HEAD                32170 non-null  object 
 14  BODY                32170 non-null  object 
 15  LEG                 32170 non-null  object 
 16  DIST

**drop original columns + columns with %**

In [30]:
cols_to_del = cols_to_split + ['SIG.STR. %', 'TD %']
df_fight_stats = df_fight_stats.drop(cols_to_del, axis = 1)

In [31]:
df_fight_stats.isna().any()

EVENT                 False
BOUT                  False
ROUND                 False
FIGHTER               False
KD                    False
SUB.ATT               False
REV.                  False
CTRL                  False
SIG.STR._landed       False
SIG.STR._attemps      False
TOTAL STR._landed     False
TOTAL STR._attemps    False
TD_landed             False
TD_attemps            False
HEAD_landed           False
HEAD_attemps          False
BODY_landed           False
BODY_attemps          False
LEG_landed            False
LEG_attemps           False
DISTANCE_landed       False
DISTANCE_attemps      False
CLINCH_landed         False
CLINCH_attemps        False
GROUND_landed         False
GROUND_attemps        False
dtype: bool

In [32]:
df_fight_stats.head(3)

,EVENT,BOUT,ROUND,FIGHTER,KD,SUB.ATT,REV.,CTRL,SIG.STR._landed,SIG.STR._attemps,...,BODY_landed,BODY_attemps,LEG_landed,LEG_attemps,DISTANCE_landed,DISTANCE_attemps,CLINCH_landed,CLINCH_attemps,GROUND_landed,GROUND_attemps
0,UFC Fight Night: Cannonier vs. Strickland,Jared Cannonier vs. Sean Strickland,Round 1,Jared Cannonier,0.0,0.0,0.0,0:00,16,43,...,7,8,6,9,16,43,0,0,0,0
1,UFC Fight Night: Cannonier vs. Strickland,Jared Cannonier vs. Sean Strickland,Round 2,Jared Cannonier,0.0,0.0,0.0,0:00,25,58,...,11,18,8,12,25,58,0,0,0,0
2,UFC Fight Night: Cannonier vs. Strickland,Jared Cannonier vs. Sean Strickland,Round 3,Jared Cannonier,0.0,0.0,0.0,0:00,33,69,...,15,21,6,8,33,69,0,0,0,0


**create agg df**

In [33]:
df_fight_stats_agg = df_fight_stats.copy()

In [34]:
df_fight_stats_agg = df_fight_stats_agg.drop(['ROUND'], axis = 1)

In [35]:
df_fight_stats_agg.head(10)

,EVENT,BOUT,FIGHTER,KD,SUB.ATT,REV.,CTRL,SIG.STR._landed,SIG.STR._attemps,TOTAL STR._landed,...,BODY_landed,BODY_attemps,LEG_landed,LEG_attemps,DISTANCE_landed,DISTANCE_attemps,CLINCH_landed,CLINCH_attemps,GROUND_landed,GROUND_attemps
0,UFC Fight Night: Cannonier vs. Strickland,Jared Cannonier vs. Sean Strickland,Jared Cannonier,0.0,0.0,0.0,0:00,16,43,16,...,7,8,6,9,16,43,0,0,0,0
1,UFC Fight Night: Cannonier vs. Strickland,Jared Cannonier vs. Sean Strickland,Jared Cannonier,0.0,0.0,0.0,0:00,25,58,25,...,11,18,8,12,25,58,0,0,0,0
2,UFC Fight Night: Cannonier vs. Strickland,Jared Cannonier vs. Sean Strickland,Jared Cannonier,0.0,0.0,0.0,0:00,33,69,33,...,15,21,6,8,33,69,0,0,0,0
3,UFC Fight Night: Cannonier vs. Strickland,Jared Cannonier vs. Sean Strickland,Jared Cannonier,0.0,0.0,0.0,0:00,27,61,27,...,11,18,2,4,27,61,0,0,0,0
4,UFC Fight Night: Cannonier vs. Strickland,Jared Cannonier vs. Sean Strickland,Jared Cannonier,0.0,0.0,0.0,0:00,40,79,40,...,16,23,2,2,40,79,0,0,0,0
5,UFC Fight Night: Cannonier vs. Strickland,Jared Cannonier vs. Sean Strickland,Sean Strickland,0.0,0.0,0.0,0:43,13,48,17,...,4,5,0,0,13,48,0,0,0,0
6,UFC Fight Night: Cannonier vs. Strickland,Jared Cannonier vs. Sean Strickland,Sean Strickland,0.0,0.0,0.0,0:00,28,73,28,...,5,8,0,1,28,73,0,0,0,0
7,UFC Fight Night: Cannonier vs. Strickland,Jared Cannonier vs. Sean Strickland,Sean Strickland,0.0,0.0,0.0,0:00,35,81,35,...,5,6,2,2,35,81,0,0,0,0
8,UFC Fight Night: Cannonier vs. Strickland,Jared Cannonier vs. Sean Strickland,Sean Strickland,0.0,0.0,0.0,0:05,33,87,34,...,4,4,0,0,33,87,0,0,0,0
9,UFC Fight Night: Cannonier vs. Strickland,Jared Cannonier vs. Sean Strickland,Sean Strickland,0.0,0.0,0.0,0:00,43,111,43,...,6,9,0,0,43,111,0,0,0,0


**handle control time value** 

In [36]:
df_fight_stats_agg['CTRL'] = df_fight_stats_agg['CTRL'].replace("--", "0:00")

In [37]:
from datetime import datetime, timedelta

In [38]:
df_fight_stats_agg['CTRL'] = df_fight_stats_agg['CTRL'].apply(lambda x: (datetime.strptime(x, '%M:%S') - datetime(1900,1,1)).total_seconds()).astype(int)

In [39]:
df_fight_stats_agg['CTRL'].describe()

count    32170.000000
mean        56.374821
std         74.226721
min          0.000000
25%          0.000000
50%         20.000000
75%         90.000000
max        300.000000
Name: CTRL, dtype: float64

In [40]:
df_fight_stats_agg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32170 entries, 0 to 32211
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   EVENT               32170 non-null  object 
 1   BOUT                32170 non-null  object 
 2   FIGHTER             32170 non-null  object 
 3   KD                  32170 non-null  float64
 4   SUB.ATT             32170 non-null  float64
 5   REV.                32170 non-null  float64
 6   CTRL                32170 non-null  int32  
 7   SIG.STR._landed     32170 non-null  int32  
 8   SIG.STR._attemps    32170 non-null  int32  
 9   TOTAL STR._landed   32170 non-null  int32  
 10  TOTAL STR._attemps  32170 non-null  int32  
 11  TD_landed           32170 non-null  int32  
 12  TD_attemps          32170 non-null  int32  
 13  HEAD_landed         32170 non-null  int32  
 14  HEAD_attemps        32170 non-null  int32  
 15  BODY_landed         32170 non-null  int32  
 16  BODY

**AGG df by bout**

In [41]:
df_total = df_fight_stats_agg.groupby(['EVENT','BOUT','FIGHTER'],as_index=False).sum()

In [42]:
df_total['BOUT'] = df_total['BOUT'].str.strip()

In [43]:
df_total['BOUT']

0         Jason MacDonald vs. Ed Herman
1         Jason MacDonald vs. Ed Herman
2         Kendall Grove vs. Chris Price
3         Kendall Grove vs. Chris Price
4         Marcus Davis vs. Forrest Petz
                      ...              
13777    Rory Markham vs. Brodie Farber
13778    Shannon Gugerty vs. Dale Hartt
13779    Shannon Gugerty vs. Dale Hartt
13780    Tim Credeur vs. Cale Yarbrough
13781    Tim Credeur vs. Cale Yarbrough
Name: BOUT, Length: 13782, dtype: object

**split bout to fighter oponnent columns**

In [44]:
df_total['OPPONENT'] = df_total.apply(lambda row: row['BOUT'].split(' vs. ')[0] if row['FIGHTER'] == row['BOUT'].split(' vs. ')[1] else row['BOUT'].split(' vs. ')[1], axis=1)

In [45]:
df_total.shape

(13782, 26)

**přidání datumu k eventu a vypočítání věku fightera**

In [46]:
df_total = df_total.merge(df_event_details[['EVENT','DATE']],how='left', left_on='EVENT', right_on='EVENT')
df_total['DATE'] = df_total['DATE'].apply(pd.to_datetime)

In [47]:
df_total.shape

(13782, 27)

In [48]:
df_fighter_tott['DOB'] = df_fighter_tott['DOB'] = pd.to_datetime(df_fighter_tott['DOB'], errors='coerce')

**přidání parametrů fightera**

In [49]:
df_fighter_tott

,FIGHTER,HEIGHT,WEIGHT,REACH,STANCE,DOB,URL,Has_fight
1,Danny Abbadi,71.0,155.0,NaN,Orthodox,1983-07-03,http://ufcstats.com/fighter-details/15df64c02b...,True
2,David Abbott,72.0,265.0,NaN,Switch,NaT,http://ufcstats.com/fighter-details/b361180739...,True
3,Shamil Abdurakhimov,75.0,235.0,76.0,Orthodox,1981-09-02,http://ufcstats.com/fighter-details/2f5cbecbbe...,True
5,Daichi Abe,71.0,170.0,71.0,Orthodox,1991-11-27,http://ufcstats.com/fighter-details/5140122c3e...,True
6,Papy Abedi,71.0,185.0,NaN,Southpaw,1978-06-30,http://ufcstats.com/fighter-details/c9f6385af6...,True
...,...,...,...,...,...,...,...,...
3901,William Gomis,NaN,145.0,NaN,NaN,1997-06-13,http://ufcstats.com/fighter-details/e1d40e8782...,True
3905,Don Shainis,66.0,145.0,NaN,NaN,1990-09-16,http://ufcstats.com/fighter-details/da4628f173...,True
3909,Lucas Alexander,70.0,145.0,NaN,NaN,1995-07-28,http://ufcstats.com/fighter-details/11583163f4...,True
3912,Carlos Mota,66.0,125.0,NaN,NaN,1995-04-15,http://ufcstats.com/fighter-details/b15862fcd6...,True


In [50]:
df_total = df_total.merge(df_fighter_tott[['FIGHTER', 'HEIGHT', 'WEIGHT', 'REACH', 'STANCE','DOB']], left_on='FIGHTER', right_on='FIGHTER', how='left')

In [51]:
#df_total = df_total.merge(df_fighter_tott[['FIGHTER', 'HEIGHT', 'WEIGHT', 'REACH', 'STANCE','DOB']], left_on='OPPONENT', right_on='FIGHTER', suffixes=('_fighter','_opponent'))

In [52]:
df_total.shape

(13782, 32)

In [53]:
df_total['AGE'] = (pd.to_datetime(df_total['DATE']) - pd.to_datetime(df_total['DOB'])).astype('<m8[Y]')
#df_total['AGE_opponent'] = (pd.to_datetime(df_total['DATE']) - pd.to_datetime(df_total['DOB_opponent'])).astype('<m8[Y]')

In [54]:
#df_total = df_total.drop(['FIGHTER_opponent', 'DOB_fighter', 'DOB_opponent'],axis=1)
df_total = df_total.drop(['DOB'],axis=1)

In [55]:
df_total = df_total.sort_values(by=['DATE', 'BOUT'], ascending=True).reset_index(drop=True)

In [56]:
list_fighters = list(df_total['FIGHTER'])
#list_fighters = list(df_total['FIGHTER_fighter'])

**vypočítaní kolikátý zápas je to pro fightera**

In [57]:
#fight_no_fighter = [list_fighters[0:i].count(fighter)+1 for i, fighter in enumerate(list(df_total['FIGHTER_fighter']))]
#fight_no_opponent = [list_fighters[0:i].count(fighter)+1 for i, fighter in enumerate(list(df_total['OPPONENT']))]
fightNo_fighter = [list_fighters[0:i].count(fighter)+1 for i, fighter in enumerate(list(df_total['FIGHTER']))]

In [58]:
df_total['fightNo'] = fightNo_fighter
#df_total['fight_no_opponent'] = fight_no_opponent

In [59]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13782 entries, 0 to 13781
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   EVENT               13782 non-null  object        
 1   BOUT                13782 non-null  object        
 2   FIGHTER             13782 non-null  object        
 3   KD                  13782 non-null  float64       
 4   SUB.ATT             13782 non-null  float64       
 5   REV.                13782 non-null  float64       
 6   CTRL                13782 non-null  int32         
 7   SIG.STR._landed     13782 non-null  int32         
 8   SIG.STR._attemps    13782 non-null  int32         
 9   TOTAL STR._landed   13782 non-null  int32         
 10  TOTAL STR._attemps  13782 non-null  int32         
 11  TD_landed           13782 non-null  int32         
 12  TD_attemps          13782 non-null  int32         
 13  HEAD_landed         13782 non-null  int32     

In [60]:
df_total

,EVENT,BOUT,FIGHTER,KD,SUB.ATT,REV.,CTRL,SIG.STR._landed,SIG.STR._attemps,TOTAL STR._landed,...,GROUND_landed,GROUND_attemps,OPPONENT,DATE,HEIGHT,WEIGHT,REACH,STANCE,AGE,fightNo
0,UFC 2: No Way Out,Frank Hamaker vs. Thaddeus Luster,Frank Hamaker,0.0,3.0,1.0,0,2,3,14,...,1,2,Thaddeus Luster,1994-03-11,NaN,NaN,NaN,NaN,NaN,1
1,UFC 2: No Way Out,Frank Hamaker vs. Thaddeus Luster,Thaddeus Luster,0.0,0.0,0.0,0,0,0,0,...,0,0,Frank Hamaker,1994-03-11,75.0,210.0,NaN,NaN,NaN,1
2,UFC 2: No Way Out,Jason DeLucia vs. Scott Baker,Jason DeLucia,0.0,5.0,4.0,0,3,5,20,...,1,1,Scott Baker,1994-03-11,71.0,190.0,NaN,Southpaw,24.0,1
3,UFC 2: No Way Out,Jason DeLucia vs. Scott Baker,Scott Baker,0.0,0.0,2.0,0,0,2,14,...,0,0,Jason DeLucia,1994-03-11,72.0,210.0,NaN,NaN,NaN,1
4,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,David Levicki,0.0,0.0,0.0,0,4,5,95,...,1,1,Johnny Rhodes,1994-03-11,77.0,275.0,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13777,UFC Fight Night: Cannonier vs. Strickland,Rafa Garcia vs. Maheshate,Rafa Garcia,0.0,0.0,0.0,370,49,98,79,...,10,21,Maheshate,2022-12-17,67.0,155.0,70.0,Orthodox,28.0,6
13778,UFC Fight Night: Cannonier vs. Strickland,Said Nurmagomedov vs. Saidyokub Kakhramonov,Said Nurmagomedov,0.0,3.0,0.0,25,7,18,21,...,0,0,Saidyokub Kakhramonov,2022-12-17,68.0,135.0,70.0,Orthodox,30.0,7
13779,UFC Fight Night: Cannonier vs. Strickland,Said Nurmagomedov vs. Saidyokub Kakhramonov,Saidyokub Kakhramonov,0.0,1.0,0.0,472,11,30,32,...,6,19,Said Nurmagomedov,2022-12-17,68.0,135.0,69.0,Orthodox,27.0,3
13780,UFC Fight Night: Cannonier vs. Strickland,Sergey Morozov vs. Journey Newson,Journey Newson,0.0,0.0,0.0,0,26,106,49,...,0,0,Sergey Morozov,2022-12-17,65.0,135.0,67.0,Orthodox,33.0,5


In [61]:
df_total.columns

Index(['EVENT', 'BOUT', 'FIGHTER', 'KD', 'SUB.ATT', 'REV.', 'CTRL',
       'SIG.STR._landed', 'SIG.STR._attemps', 'TOTAL STR._landed',
       'TOTAL STR._attemps', 'TD_landed', 'TD_attemps', 'HEAD_landed',
       'HEAD_attemps', 'BODY_landed', 'BODY_attemps', 'LEG_landed',
       'LEG_attemps', 'DISTANCE_landed', 'DISTANCE_attemps', 'CLINCH_landed',
       'CLINCH_attemps', 'GROUND_landed', 'GROUND_attemps', 'OPPONENT', 'DATE',
       'HEIGHT', 'WEIGHT', 'REACH', 'STANCE', 'AGE', 'fightNo'],
      dtype='object')

In [62]:
df_total = df_total.merge(df_total,how='left', left_on=['EVENT', 'BOUT', 'OPPONENT'], right_on=['EVENT', 'BOUT', 'FIGHTER'], suffixes=('_fighter', '_opponent'))

In [63]:
df_total = df_total.drop(['FIGHTER_opponent', 'OPPONENT_opponent', 'DATE_opponent'],axis=1)

In [64]:
df_total= df_total.rename(columns={'BOUT_fighter': "BOUT", "FIGHTER_fighter": "FIGHTER", "OPPONENT_fighter": "OPPONENT"})

In [65]:
df_total.shape

(13782, 61)

In [66]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13782 entries, 0 to 13781
Data columns (total 61 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   EVENT                        13782 non-null  object        
 1   BOUT                         13782 non-null  object        
 2   FIGHTER                      13782 non-null  object        
 3   KD_fighter                   13782 non-null  float64       
 4   SUB.ATT_fighter              13782 non-null  float64       
 5   REV._fighter                 13782 non-null  float64       
 6   CTRL_fighter                 13782 non-null  int32         
 7   SIG.STR._landed_fighter      13782 non-null  int32         
 8   SIG.STR._attemps_fighter     13782 non-null  int32         
 9   TOTAL STR._landed_fighter    13782 non-null  int32         
 10  TOTAL STR._attemps_fighter   13782 non-null  int32         
 11  TD_landed_fighter            13782 non-nu

In [67]:
#handle_result_df(df_results)

In [68]:
df_results.head()

,EVENT,BOUT,OUTCOME,WEIGHTCLASS,METHOD,ROUND,TIME,TIME FORMAT,REFEREE,DETAILS,URL
0,UFC Fight Night: Cannonier vs. Strickland,Jared Cannonier vs. Sean Strickland,W/L,Middleweight Bout,Decision - Split,5,5:00,5 Rnd (5-5-5-5-5),Herb Dean,Derek Cleary 46 - 49.Sal D'amato 49 - 46.Junic...,http://ufcstats.com/fight-details/4ea48bf2407b...
1,UFC Fight Night: Cannonier vs. Strickland,Arman Tsarukyan vs. Damir Ismagulov,W/L,Lightweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Mark Smith,Mike Bell 27 - 30.Sal D'amato 27 - 30.Ron McCa...,http://ufcstats.com/fight-details/ce7744d45a69...
2,UFC Fight Night: Cannonier vs. Strickland,Amir Albazi vs. Alessandro Costa,W/L,Flyweight Bout,KO/TKO,3,2:13,3 Rnd (5-5-5),Keith Peterson,Punch to Head At Distance,http://ufcstats.com/fight-details/d95eeba3de7e...
3,UFC Fight Night: Cannonier vs. Strickland,Alex Caceres vs. Julian Erosa,W/L,Featherweight Bout,KO/TKO,1,3:04,3 Rnd (5-5-5),Herb Dean,Kick to Head At Distance,http://ufcstats.com/fight-details/9c1a2ac64c98...
4,UFC Fight Night: Cannonier vs. Strickland,Drew Dober vs. Bobby Green,W/L,Lightweight Bout,KO/TKO,2,2:45,3 Rnd (5-5-5),Keith Peterson,Punch to Head At Distance,http://ufcstats.com/fight-details/c335f44e4f35...


In [69]:
df_results['Winner'] = df_results.apply(lambda row: 'NC' if row['OUTCOME'] == 'NC/NC' else 'DRAW' if row['OUTCOME'] == 'D/D' else row['BOUT'].split(' vs. ')[(row['OUTCOME'].split('/')).index('W')], axis=1)

In [70]:
df_results.head(30)

,EVENT,BOUT,OUTCOME,WEIGHTCLASS,METHOD,ROUND,TIME,TIME FORMAT,REFEREE,DETAILS,URL,Winner
0,UFC Fight Night: Cannonier vs. Strickland,Jared Cannonier vs. Sean Strickland,W/L,Middleweight Bout,Decision - Split,5,5:00,5 Rnd (5-5-5-5-5),Herb Dean,Derek Cleary 46 - 49.Sal D'amato 49 - 46.Junic...,http://ufcstats.com/fight-details/4ea48bf2407b...,Jared Cannonier
1,UFC Fight Night: Cannonier vs. Strickland,Arman Tsarukyan vs. Damir Ismagulov,W/L,Lightweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Mark Smith,Mike Bell 27 - 30.Sal D'amato 27 - 30.Ron McCa...,http://ufcstats.com/fight-details/ce7744d45a69...,Arman Tsarukyan
2,UFC Fight Night: Cannonier vs. Strickland,Amir Albazi vs. Alessandro Costa,W/L,Flyweight Bout,KO/TKO,3,2:13,3 Rnd (5-5-5),Keith Peterson,Punch to Head At Distance,http://ufcstats.com/fight-details/d95eeba3de7e...,Amir Albazi
3,UFC Fight Night: Cannonier vs. Strickland,Alex Caceres vs. Julian Erosa,W/L,Featherweight Bout,KO/TKO,1,3:04,3 Rnd (5-5-5),Herb Dean,Kick to Head At Distance,http://ufcstats.com/fight-details/9c1a2ac64c98...,Alex Caceres
4,UFC Fight Night: Cannonier vs. Strickland,Drew Dober vs. Bobby Green,W/L,Lightweight Bout,KO/TKO,2,2:45,3 Rnd (5-5-5),Keith Peterson,Punch to Head At Distance,http://ufcstats.com/fight-details/c335f44e4f35...,Drew Dober
5,UFC Fight Night: Cannonier vs. Strickland,Cody Brundage vs. Michal Oleksiejczuk,L/W,Middleweight Bout,KO/TKO,1,3:16,3 Rnd (5-5-5),Mark Smith,Punch to Head On Ground,http://ufcstats.com/fight-details/e0444752d18f...,Michal Oleksiejczuk
6,UFC Fight Night: Cannonier vs. Strickland,Cheyanne Vlismas vs. Cory McKenna,L/W,Women's Strawweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Chris Tognoni,Derek Cleary 28 - 29.Sal D'amato 28 - 29.Junic...,http://ufcstats.com/fight-details/c960184a583f...,Cory McKenna
7,UFC Fight Night: Cannonier vs. Strickland,Jake Matthews vs. Matthew Semelsberger,L/W,Welterweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Herb Dean,Ron McCarthy 27 - 30.Mike Bell 28 - 29.Chris L...,http://ufcstats.com/fight-details/5f4e944ff1bd...,Matthew Semelsberger
8,UFC Fight Night: Cannonier vs. Strickland,Said Nurmagomedov vs. Saidyokub Kakhramonov,W/L,Bantamweight Bout,Submission,2,3:50,3 Rnd (5-5-5),Keith Peterson,Guillotine Choke On Ground,http://ufcstats.com/fight-details/ed6203e849a2...,Said Nurmagomedov
9,UFC Fight Night: Cannonier vs. Strickland,Rafa Garcia vs. Maheshate,W/L,Lightweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Mark Smith,Mike Bell 27 - 30.Derek Cleary 27 - 30.Chris L...,http://ufcstats.com/fight-details/efc64d8aae56...,Rafa Garcia


In [71]:
df_results.groupby('TIME FORMAT')['TIME FORMAT'].count().sort_values(ascending=False).head(50)

TIME FORMAT
3 Rnd (5-5-5)           6075
5 Rnd (5-5-5-5-5)        597
1 Rnd + OT (12-3)         88
No Time Limit             31
3 Rnd + OT (5-5-5-5)      26
1 Rnd (20)                25
1 Rnd + 2OT (15-3-3)      20
2 Rnd (5-5)               14
1 Rnd (15)                12
1 Rnd (10)                 7
1 Rnd (12)                 6
1 Rnd + OT (30-5)          3
1 Rnd (18)                 2
1 Rnd + OT (15-3)          2
1 Rnd + OT (27-3)          1
1 Rnd + OT (30-3)          1
1 Rnd + OT (31-5)          1
1 Rnd + 2OT (24-3-3)       1
1 Rnd (30)                 1
Name: TIME FORMAT, dtype: int64

In [72]:
WEIGHTCLASSES_my = ['Lightweight', 'Welterweight', 'Middleweight', 'Featherweight','Heavyweigh', 'Light Heavyweight', 'Bantamweight', 'Flyweight', 'Strawweight', 'Open Weight', 'Catch Weight']

In [73]:
WeightClasses_df = df_results['WEIGHTCLASS']

In [74]:
WeightClasses_df

0        Middleweight Bout
1         Lightweight Bout
2           Flyweight Bout
3       Featherweight Bout
4         Lightweight Bout
               ...        
6908      Open Weight Bout
6909      Open Weight Bout
6910      Open Weight Bout
6911      Open Weight Bout
6912      Open Weight Bout
Name: WEIGHTCLASS, Length: 6913, dtype: object

In [75]:
result=[]
women = []
Title = []
for bout in WeightClasses_df:
  bout_low = bout.lower()
  women.append(1) if 'women' in bout_low else women.append(0)
  Title.append(1) if 'title' in bout_low else Title.append(0)
  find = False
  for classes in WEIGHTCLASSES_my:   
    if classes.lower() in bout_low.lower():
      result.append(classes)
      find = True      
      break
  if not find:
    result.append('Open Weight')
    print(classes, bout)

Catch Weight Ultimate Ultimate '96 Tournament Title Bout
Catch Weight UFC 10 Tournament Title Bout
Catch Weight UFC Superfight Championship Bout
Catch Weight UFC 8 Tournament Title Bout
Catch Weight UFC Superfight Championship Bout
Catch Weight Ultimate Ultimate '95 Tournament Title Bout
Catch Weight UFC 7 Tournament Title Bout
Catch Weight UFC Superfight Championship Bout
Catch Weight UFC 6 Tournament Title Bout
Catch Weight UFC Superfight Championship Bout
Catch Weight UFC 5 Tournament Title Bout
Catch Weight UFC Superfight Championship Bout
Catch Weight UFC 4 Tournament Title Bout
Catch Weight UFC 3 Tournament Title Bout
Catch Weight UFC 2 Tournament Title Bout


In [76]:
result[:10]

['Middleweight',
 'Lightweight',
 'Flyweight',
 'Featherweight',
 'Lightweight',
 'Middleweight',
 'Strawweight',
 'Welterweight',
 'Bantamweight',
 'Lightweight']

In [77]:
df_results.loc[:,"WeightClass"] = result
df_results.loc[:,"Tittle"] = Title
df_results.loc[:,"Women_fight"] = women


In [78]:
df_results.tail()

,EVENT,BOUT,OUTCOME,WEIGHTCLASS,METHOD,ROUND,TIME,TIME FORMAT,REFEREE,DETAILS,URL,Winner,WeightClass,Tittle,Women_fight
6908,UFC 2: No Way Out,Orlando Wiet vs. Robert Lucarelli,W/L,Open Weight Bout,KO/TKO,1,2:50,No Time Limit,John McCarthy,toCorner Stoppage,http://ufcstats.com/fight-details/3b020d4914b4...,Orlando Wiet,Open Weight,0,0
6909,UFC 2: No Way Out,Frank Hamaker vs. Thaddeus Luster,W/L,Open Weight Bout,Submission,1,4:52,No Time Limit,John McCarthy,Keylock From Half Guard,http://ufcstats.com/fight-details/d917c8c7461b...,Frank Hamaker,Open Weight,0,0
6910,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,W/L,Open Weight Bout,KO/TKO,1,12:13,No Time Limit,John McCarthy,Punches to Head From GuardSubmission to Strikes,http://ufcstats.com/fight-details/ccee020be2e8...,Johnny Rhodes,Open Weight,0,0
6911,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,W/L,Open Weight Bout,Submission,1,0:58,No Time Limit,John McCarthy,Guillotine Choke Standing,http://ufcstats.com/fight-details/4b9ae533ccb3...,Patrick Smith,Open Weight,0,0
6912,UFC 2: No Way Out,Scott Morris vs. Sean Daugherty,W/L,Open Weight Bout,Submission,1,0:20,No Time Limit,John McCarthy,Guillotine Choke From Mount,http://ufcstats.com/fight-details/4acab67848e7...,Scott Morris,Open Weight,0,0


In [79]:
df_results['TimeSec'] = df_results['TIME'].apply(lambda x: (datetime.strptime(x, '%M:%S') - datetime(1900,1,1)).total_seconds()).astype(int)

In [80]:
df_results.groupby('TIME FORMAT')['TIME FORMAT'].count().sort_values(ascending=False).head(50)

TIME FORMAT
3 Rnd (5-5-5)           6075
5 Rnd (5-5-5-5-5)        597
1 Rnd + OT (12-3)         88
No Time Limit             31
3 Rnd + OT (5-5-5-5)      26
1 Rnd (20)                25
1 Rnd + 2OT (15-3-3)      20
2 Rnd (5-5)               14
1 Rnd (15)                12
1 Rnd (10)                 7
1 Rnd (12)                 6
1 Rnd + OT (30-5)          3
1 Rnd (18)                 2
1 Rnd + OT (15-3)          2
1 Rnd + OT (27-3)          1
1 Rnd + OT (30-3)          1
1 Rnd + OT (31-5)          1
1 Rnd + 2OT (24-3-3)       1
1 Rnd (30)                 1
Name: TIME FORMAT, dtype: int64

In [81]:
import re
def extract_rounds(text):
  if text == "No Time Limit":
    return [0]
  else:
    result = re.findall(r'\((.*?)\)', text)
    result = list(map(int,result[0].split("-")))
    return result

In [82]:
df_results['TIME FORMAT'] = df_results['TIME FORMAT'].apply(lambda x: extract_rounds(x))

In [83]:
df_results

,EVENT,BOUT,OUTCOME,WEIGHTCLASS,METHOD,ROUND,TIME,TIME FORMAT,REFEREE,DETAILS,URL,Winner,WeightClass,Tittle,Women_fight,TimeSec
0,UFC Fight Night: Cannonier vs. Strickland,Jared Cannonier vs. Sean Strickland,W/L,Middleweight Bout,Decision - Split,5,5:00,"[5, 5, 5, 5, 5]",Herb Dean,Derek Cleary 46 - 49.Sal D'amato 49 - 46.Junic...,http://ufcstats.com/fight-details/4ea48bf2407b...,Jared Cannonier,Middleweight,0,0,300
1,UFC Fight Night: Cannonier vs. Strickland,Arman Tsarukyan vs. Damir Ismagulov,W/L,Lightweight Bout,Decision - Unanimous,3,5:00,"[5, 5, 5]",Mark Smith,Mike Bell 27 - 30.Sal D'amato 27 - 30.Ron McCa...,http://ufcstats.com/fight-details/ce7744d45a69...,Arman Tsarukyan,Lightweight,0,0,300
2,UFC Fight Night: Cannonier vs. Strickland,Amir Albazi vs. Alessandro Costa,W/L,Flyweight Bout,KO/TKO,3,2:13,"[5, 5, 5]",Keith Peterson,Punch to Head At Distance,http://ufcstats.com/fight-details/d95eeba3de7e...,Amir Albazi,Flyweight,0,0,133
3,UFC Fight Night: Cannonier vs. Strickland,Alex Caceres vs. Julian Erosa,W/L,Featherweight Bout,KO/TKO,1,3:04,"[5, 5, 5]",Herb Dean,Kick to Head At Distance,http://ufcstats.com/fight-details/9c1a2ac64c98...,Alex Caceres,Featherweight,0,0,184
4,UFC Fight Night: Cannonier vs. Strickland,Drew Dober vs. Bobby Green,W/L,Lightweight Bout,KO/TKO,2,2:45,"[5, 5, 5]",Keith Peterson,Punch to Head At Distance,http://ufcstats.com/fight-details/c335f44e4f35...,Drew Dober,Lightweight,0,0,165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6908,UFC 2: No Way Out,Orlando Wiet vs. Robert Lucarelli,W/L,Open Weight Bout,KO/TKO,1,2:50,[0],John McCarthy,toCorner Stoppage,http://ufcstats.com/fight-details/3b020d4914b4...,Orlando Wiet,Open Weight,0,0,170
6909,UFC 2: No Way Out,Frank Hamaker vs. Thaddeus Luster,W/L,Open Weight Bout,Submission,1,4:52,[0],John McCarthy,Keylock From Half Guard,http://ufcstats.com/fight-details/d917c8c7461b...,Frank Hamaker,Open Weight,0,0,292
6910,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,W/L,Open Weight Bout,KO/TKO,1,12:13,[0],John McCarthy,Punches to Head From GuardSubmission to Strikes,http://ufcstats.com/fight-details/ccee020be2e8...,Johnny Rhodes,Open Weight,0,0,733
6911,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,W/L,Open Weight Bout,Submission,1,0:58,[0],John McCarthy,Guillotine Choke Standing,http://ufcstats.com/fight-details/4b9ae533ccb3...,Patrick Smith,Open Weight,0,0,58


In [84]:
df_results['TIME FORMAT']

0       [5, 5, 5, 5, 5]
1             [5, 5, 5]
2             [5, 5, 5]
3             [5, 5, 5]
4             [5, 5, 5]
             ...       
6908                [0]
6909                [0]
6910                [0]
6911                [0]
6912                [0]
Name: TIME FORMAT, Length: 6913, dtype: object

In [85]:
df_results['TotalTime'] = df_results.apply(lambda row: sum(row['TIME FORMAT'][0:int(row['ROUND'])-1])*60 + row['TimeSec'],axis=1)

In [86]:
df_results['ROUND'].unique()

array([5, 3, 1, 2, 4], dtype=int64)

In [87]:
df_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6913 entries, 0 to 6912
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   EVENT        6913 non-null   object
 1   BOUT         6913 non-null   object
 2   OUTCOME      6913 non-null   object
 3   WEIGHTCLASS  6913 non-null   object
 4   METHOD       6913 non-null   object
 5   ROUND        6913 non-null   int64 
 6   TIME         6913 non-null   object
 7   TIME FORMAT  6913 non-null   object
 8   REFEREE      6881 non-null   object
 9   DETAILS      6840 non-null   object
 10  URL          6913 non-null   object
 11  Winner       6913 non-null   object
 12  WeightClass  6913 non-null   object
 13  Tittle       6913 non-null   int64 
 14  Women_fight  6913 non-null   int64 
 15  TimeSec      6913 non-null   int32 
 16  TotalTime    6913 non-null   int64 
dtypes: int32(1), int64(4), object(12)
memory usage: 891.3+ KB


In [88]:
df_results = df_results.drop(['OUTCOME','WEIGHTCLASS','TimeSec'],axis=1)

In [89]:
df_results.columns

Index(['EVENT', 'BOUT', 'METHOD', 'ROUND', 'TIME', 'TIME FORMAT', 'REFEREE',
       'DETAILS', 'URL', 'Winner', 'WeightClass', 'Tittle', 'Women_fight',
       'TotalTime'],
      dtype='object')

In [90]:
df_event_details

,EVENT,URL,DATE,LOCATION
0,UFC Fight Night: Cannonier vs. Strickland,http://ufcstats.com/event-details/56ec58954158...,"December 17, 2022","Las Vegas, Nevada, USA"
1,UFC 282: Blachowicz vs. Ankalaev,http://ufcstats.com/event-details/f65a0eb902f9...,"December 10, 2022","Las Vegas, Nevada, USA"
2,UFC Fight Night: Thompson vs. Holland,http://ufcstats.com/event-details/b23388ff8ac6...,"December 03, 2022","Orlando, Florida, USA"
3,UFC Fight Night: Nzechukwu vs. Cutelaba,http://ufcstats.com/event-details/012fc7cd0779...,"November 19, 2022","Las Vegas, Nevada, USA"
4,UFC 281: Adesanya vs. Pereira,http://ufcstats.com/event-details/b3b6e80b7d5f...,"November 12, 2022","New York City, New York, USA"
...,...,...,...,...
625,UFC 6: Clash of the Titans,http://ufcstats.com/event-details/1c3f5e85b59e...,"July 14, 1995","Casper, Wyoming, USA"
626,UFC 5: The Return of the Beast,http://ufcstats.com/event-details/dedc3bb440d0...,"April 07, 1995","Charlotte, North Carolina, USA"
627,UFC 4: Revenge of the Warriors,http://ufcstats.com/event-details/b60391da771d...,"December 16, 1994","Tulsa, Oklahoma, USA"
628,UFC 3: The American Dream,http://ufcstats.com/event-details/1a49e0670dfa...,"September 09, 1994","Charlotte, North Carolina, USA"


In [91]:
df_results

,EVENT,BOUT,METHOD,ROUND,TIME,TIME FORMAT,REFEREE,DETAILS,URL,Winner,WeightClass,Tittle,Women_fight,TotalTime
0,UFC Fight Night: Cannonier vs. Strickland,Jared Cannonier vs. Sean Strickland,Decision - Split,5,5:00,"[5, 5, 5, 5, 5]",Herb Dean,Derek Cleary 46 - 49.Sal D'amato 49 - 46.Junic...,http://ufcstats.com/fight-details/4ea48bf2407b...,Jared Cannonier,Middleweight,0,0,1500
1,UFC Fight Night: Cannonier vs. Strickland,Arman Tsarukyan vs. Damir Ismagulov,Decision - Unanimous,3,5:00,"[5, 5, 5]",Mark Smith,Mike Bell 27 - 30.Sal D'amato 27 - 30.Ron McCa...,http://ufcstats.com/fight-details/ce7744d45a69...,Arman Tsarukyan,Lightweight,0,0,900
2,UFC Fight Night: Cannonier vs. Strickland,Amir Albazi vs. Alessandro Costa,KO/TKO,3,2:13,"[5, 5, 5]",Keith Peterson,Punch to Head At Distance,http://ufcstats.com/fight-details/d95eeba3de7e...,Amir Albazi,Flyweight,0,0,733
3,UFC Fight Night: Cannonier vs. Strickland,Alex Caceres vs. Julian Erosa,KO/TKO,1,3:04,"[5, 5, 5]",Herb Dean,Kick to Head At Distance,http://ufcstats.com/fight-details/9c1a2ac64c98...,Alex Caceres,Featherweight,0,0,184
4,UFC Fight Night: Cannonier vs. Strickland,Drew Dober vs. Bobby Green,KO/TKO,2,2:45,"[5, 5, 5]",Keith Peterson,Punch to Head At Distance,http://ufcstats.com/fight-details/c335f44e4f35...,Drew Dober,Lightweight,0,0,465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6908,UFC 2: No Way Out,Orlando Wiet vs. Robert Lucarelli,KO/TKO,1,2:50,[0],John McCarthy,toCorner Stoppage,http://ufcstats.com/fight-details/3b020d4914b4...,Orlando Wiet,Open Weight,0,0,170
6909,UFC 2: No Way Out,Frank Hamaker vs. Thaddeus Luster,Submission,1,4:52,[0],John McCarthy,Keylock From Half Guard,http://ufcstats.com/fight-details/d917c8c7461b...,Frank Hamaker,Open Weight,0,0,292
6910,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,KO/TKO,1,12:13,[0],John McCarthy,Punches to Head From GuardSubmission to Strikes,http://ufcstats.com/fight-details/ccee020be2e8...,Johnny Rhodes,Open Weight,0,0,733
6911,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,Submission,1,0:58,[0],John McCarthy,Guillotine Choke Standing,http://ufcstats.com/fight-details/4b9ae533ccb3...,Patrick Smith,Open Weight,0,0,58


In [92]:
df_results = df_results.merge(df_event_details[['EVENT','DATE']],how='left', left_on='EVENT', right_on='EVENT')

In [93]:
df_results = df_results[['DATE','EVENT', 'BOUT', 'Winner', 'WeightClass', 'Tittle', 'Women_fight','METHOD','TIME FORMAT', 'ROUND', 'TIME', 'TotalTime', 'REFEREE',
       'DETAILS', 'URL']]

In [94]:
df_results.head()

,DATE,EVENT,BOUT,Winner,WeightClass,Tittle,Women_fight,METHOD,TIME FORMAT,ROUND,TIME,TotalTime,REFEREE,DETAILS,URL
0,NaN,UFC Fight Night: Cannonier vs. Strickland,Jared Cannonier vs. Sean Strickland,Jared Cannonier,Middleweight,0,0,Decision - Split,"[5, 5, 5, 5, 5]",5,5:00,1500,Herb Dean,Derek Cleary 46 - 49.Sal D'amato 49 - 46.Junic...,http://ufcstats.com/fight-details/4ea48bf2407b...
1,NaN,UFC Fight Night: Cannonier vs. Strickland,Arman Tsarukyan vs. Damir Ismagulov,Arman Tsarukyan,Lightweight,0,0,Decision - Unanimous,"[5, 5, 5]",3,5:00,900,Mark Smith,Mike Bell 27 - 30.Sal D'amato 27 - 30.Ron McCa...,http://ufcstats.com/fight-details/ce7744d45a69...
2,NaN,UFC Fight Night: Cannonier vs. Strickland,Amir Albazi vs. Alessandro Costa,Amir Albazi,Flyweight,0,0,KO/TKO,"[5, 5, 5]",3,2:13,733,Keith Peterson,Punch to Head At Distance,http://ufcstats.com/fight-details/d95eeba3de7e...
3,NaN,UFC Fight Night: Cannonier vs. Strickland,Alex Caceres vs. Julian Erosa,Alex Caceres,Featherweight,0,0,KO/TKO,"[5, 5, 5]",1,3:04,184,Herb Dean,Kick to Head At Distance,http://ufcstats.com/fight-details/9c1a2ac64c98...
4,NaN,UFC Fight Night: Cannonier vs. Strickland,Drew Dober vs. Bobby Green,Drew Dober,Lightweight,0,0,KO/TKO,"[5, 5, 5]",2,2:45,465,Keith Peterson,Punch to Head At Distance,http://ufcstats.com/fight-details/c335f44e4f35...


In [95]:
df_results['METHOD'].unique()

array(['Decision - Split ', 'Decision - Unanimous ', 'KO/TKO ',
       'Submission ', 'Decision - Majority ', "TKO - Doctor's Stoppage ",
       'Could Not Continue ', 'DQ ', 'Other ', 'Overturned '],
      dtype=object)

In [96]:
df_results[df_results['METHOD']=='KO/TKO ']['DETAILS'].unique()

array(['Punch to Head At Distance ', 'Kick to Head At Distance ',
       'Punch to Head On Ground ', 'Punches to Head At Distance ',
       'Punches to Head On Ground ', 'Punch to Head In Clinch ',
       'toCorner Stoppage', 'Punches to Head From Half Guard ',
       'Punches to Head From Guard ', 'Knee to Head At Distance ',
       'Kick to Head At DistanceWheel Kick', 'Spinning Back Kick Head ',
       'Kick to Body At DistanceFront Kick',
       'Elbows to Head From Half Guard ', 'Flying Knee Body ',
       'to At DistanceKnee injury', 'to At Distance ',
       'Kick to Leg At Distance ', 'Elbows to Head From Mount ',
       'Knee to Head In Clinch ', 'Flying Knee Head ',
       'Punches to Head From Mount ',
       'Punches to Head From Back Control ', 'Kick to Body From Bottom ',
       'Punches to Head In Clinch ', 'Kick to Body At Distance ',
       'Punch to Body At Distance ', 'Elbows to Head On Ground ',
       'Knees to Head In Clinch ', 'toKnee Injury',
       'Spinning Ba

In [97]:
df_results[df_results['DETAILS']=='toSubmission to Strikes']

,DATE,EVENT,BOUT,Winner,WeightClass,Tittle,Women_fight,METHOD,TIME FORMAT,ROUND,TIME,TotalTime,REFEREE,DETAILS,URL
6860,NaN,UFC 7: The Brawl in Buffalo,Onassis Parungao vs. Francesco Maturi,Onassis Parungao,Open Weight,0,0,KO/TKO,[20],1,5:26,326,John McCarthy,toSubmission to Strikes,http://ufcstats.com/fight-details/e4fe950846b5...


In [98]:
df_results[df_results['BOUT'].str.contains('Silva')]


,DATE,EVENT,BOUT,Winner,WeightClass,Tittle,Women_fight,METHOD,TIME FORMAT,ROUND,TIME,TotalTime,REFEREE,DETAILS,URL
23,NaN,UFC 282: Blachowicz vs. Ankalaev,TJ Brown vs. Erik Silva,TJ Brown,Featherweight,0,0,Submission,"[5, 5, 5]",3,3:41,821,Mark Smith,Arm Triangle From Side Control,http://ufcstats.com/fight-details/2d9d966e0e8d...
49,NaN,UFC Fight Night: Nzechukwu vs. Cutelaba,Natalia Silva vs. Tereza Bleda,Natalia Silva,Flyweight,0,1,KO/TKO,"[5, 5, 5]",3,1:27,687,Chris Tognoni,Spinning Back Kick Head,http://ufcstats.com/fight-details/33719a2d9c69...
60,NaN,UFC 281: Adesanya vs. Pereira,Karolina Kowalkiewicz vs. Silvana Gomez Juarez,Karolina Kowalkiewicz,Strawweight,0,1,Decision - Unanimous,"[5, 5, 5]",3,5:00,900,Marc Goddard,Mike Bell 28 - 29.Sal D'amato 28 - 29.Dave Tir...,http://ufcstats.com/fight-details/ff2e4b411f69...
104,NaN,UFC Fight Night: Dern vs. Yan,Joaquim Silva vs. Jesse Ronson,Joaquim Silva,Lightweight,0,0,KO/TKO,"[5, 5, 5]",2,3:08,488,Keith Peterson,Flying Knee Head,http://ufcstats.com/fight-details/ecc56a86a1ee...
157,NaN,UFC 278: Usman vs. Edwards,Daniel Da Silva vs. Victor Altamirano,Victor Altamirano,Flyweight,0,0,KO/TKO,"[5, 5, 5]",1,3:39,219,Dave Seljestad,Punches to Head From Guard,http://ufcstats.com/fight-details/06d6cde74797...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6344,NaN,UFC 60: Hughes vs Gracie,Brandon Vera vs. Assuerio Silva,Brandon Vera,Heavyweigh,0,0,Submission,"[5, 5, 5]",1,2:39,159,John McCarthy,Guillotine Choke From Bottom Guard,http://ufcstats.com/fight-details/c66bd50ed1e7...
6385,NaN,UFC Fight Night 3,Tim Sylvia vs. Assuerio Silva,Tim Sylvia,Heavyweigh,0,0,Decision - Unanimous,"[5, 5, 5]",3,5:00,900,John McCarthy,Adalaide Byrd 27 - 30.Cecil Peoples 27 - 30.Da...,http://ufcstats.com/fight-details/537012aa6fb2...
6676,NaN,UFC 25: Ultimate Japan 3,Tito Ortiz vs. Wanderlei Silva,Tito Ortiz,Heavyweigh,1,0,Decision - Unanimous,"[5, 5, 5, 5, 5]",5,5:00,1500,John McCarthy,47 - 49.46 - 49.46 - 49.,http://ufcstats.com/fight-details/0adfdf8f563b...
6715,NaN,UFC 20: Battle for the Gold,Wanderlei Silva vs. Tony Petarra,Wanderlei Silva,Middleweight,0,0,KO/TKO,"[12, 3]",1,2:53,173,Mario Yamasaki,Knees to Head In Clinch,http://ufcstats.com/fight-details/c892b23ad8f2...


In [99]:
df_total

,EVENT,BOUT,FIGHTER,KD_fighter,SUB.ATT_fighter,REV._fighter,CTRL_fighter,SIG.STR._landed_fighter,SIG.STR._attemps_fighter,TOTAL STR._landed_fighter,...,CLINCH_landed_opponent,CLINCH_attemps_opponent,GROUND_landed_opponent,GROUND_attemps_opponent,HEIGHT_opponent,WEIGHT_opponent,REACH_opponent,STANCE_opponent,AGE_opponent,fightNo_opponent
0,UFC 2: No Way Out,Frank Hamaker vs. Thaddeus Luster,Frank Hamaker,0.0,3.0,1.0,0,2,3,14,...,0.0,0.0,0.0,0.0,75.0,210.0,NaN,NaN,NaN,1.0
1,UFC 2: No Way Out,Frank Hamaker vs. Thaddeus Luster,Thaddeus Luster,0.0,0.0,0.0,0,0,0,0,...,0.0,0.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,1.0
2,UFC 2: No Way Out,Jason DeLucia vs. Scott Baker,Jason DeLucia,0.0,5.0,4.0,0,3,5,20,...,0.0,0.0,0.0,0.0,72.0,210.0,NaN,NaN,NaN,1.0
3,UFC 2: No Way Out,Jason DeLucia vs. Scott Baker,Scott Baker,0.0,0.0,2.0,0,0,2,14,...,0.0,0.0,1.0,1.0,71.0,190.0,NaN,Southpaw,24.0,1.0
4,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,David Levicki,0.0,0.0,0.0,0,4,5,95,...,1.0,1.0,9.0,15.0,72.0,210.0,NaN,Orthodox,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13777,UFC Fight Night: Cannonier vs. Strickland,Rafa Garcia vs. Maheshate,Rafa Garcia,0.0,0.0,0.0,370,49,98,79,...,1.0,2.0,1.0,1.0,72.0,155.0,71.0,Orthodox,23.0,2.0
13778,UFC Fight Night: Cannonier vs. Strickland,Said Nurmagomedov vs. Saidyokub Kakhramonov,Said Nurmagomedov,0.0,3.0,0.0,25,7,18,21,...,0.0,1.0,6.0,19.0,68.0,135.0,69.0,Orthodox,27.0,3.0
13779,UFC Fight Night: Cannonier vs. Strickland,Said Nurmagomedov vs. Saidyokub Kakhramonov,Saidyokub Kakhramonov,0.0,1.0,0.0,472,11,30,32,...,4.0,6.0,0.0,0.0,68.0,135.0,70.0,Orthodox,30.0,7.0
13780,UFC Fight Night: Cannonier vs. Strickland,Sergey Morozov vs. Journey Newson,Journey Newson,0.0,0.0,0.0,0,26,106,49,...,9.0,11.0,5.0,6.0,66.0,135.0,67.0,Orthodox,33.0,5.0


In [100]:
df_results['BOUT'] = df_results['BOUT'].str.split().str.join(' ')
df_results['EVENT'] = df_results['EVENT'].str.split().str.join(' ')

In [101]:
df_total = df_total.merge(df_results, on=['EVENT','BOUT'], how='left')

In [102]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13784 entries, 0 to 13783
Data columns (total 74 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   EVENT                        13784 non-null  object        
 1   BOUT                         13784 non-null  object        
 2   FIGHTER                      13784 non-null  object        
 3   KD_fighter                   13784 non-null  float64       
 4   SUB.ATT_fighter              13784 non-null  float64       
 5   REV._fighter                 13784 non-null  float64       
 6   CTRL_fighter                 13784 non-null  int32         
 7   SIG.STR._landed_fighter      13784 non-null  int32         
 8   SIG.STR._attemps_fighter     13784 non-null  int32         
 9   TOTAL STR._landed_fighter    13784 non-null  int32         
 10  TOTAL STR._attemps_fighter   13784 non-null  int32         
 11  TD_landed_fighter            13784 non-nu

In [103]:
df_total.loc[:,'Win'] = (df_total['Winner'].str.strip() == df_total['FIGHTER'].str.strip()).astype(int)
df_total.loc[:,'DRAW'] = (df_total['Winner'].str.strip() == 'Draw').astype(int)
df_total.loc[:,'No_contest'] = (df_total['Winner'].str.strip() == 'NC').astype(int)


In [104]:
df_total= df_total[['EVENT', 'BOUT','DATE_fighter','WeightClass', 'Tittle', 'Women_fight',
       'METHOD', 'TIME FORMAT', 'ROUND', 'TIME', 'TotalTime', 'REFEREE',
       'DETAILS','URL','FIGHTER','Win', 'DRAW', 'No_contest', 'HEIGHT_fighter', 'WEIGHT_fighter', 'REACH_fighter',
       'STANCE_fighter', 'AGE_fighter', 'fightNo_fighter', 'OPPONENT', 'HEIGHT_opponent',
       'WEIGHT_opponent', 'REACH_opponent', 'STANCE_opponent', 'AGE_opponent',
       'fightNo_opponent', 'KD_fighter', 'SUB.ATT_fighter',
       'REV._fighter', 'CTRL_fighter', 'SIG.STR._landed_fighter',
       'SIG.STR._attemps_fighter', 'TOTAL STR._landed_fighter',
       'TOTAL STR._attemps_fighter', 'TD_landed_fighter', 'TD_attemps_fighter',
       'HEAD_landed_fighter', 'HEAD_attemps_fighter', 'BODY_landed_fighter',
       'BODY_attemps_fighter', 'LEG_landed_fighter', 'LEG_attemps_fighter',
       'DISTANCE_landed_fighter', 'DISTANCE_attemps_fighter',
       'CLINCH_landed_fighter', 'CLINCH_attemps_fighter',
       'GROUND_landed_fighter', 'GROUND_attemps_fighter', 'KD_opponent',
       'SUB.ATT_opponent', 'REV._opponent', 'CTRL_opponent',
       'SIG.STR._landed_opponent', 'SIG.STR._attemps_opponent',
       'TOTAL STR._landed_opponent', 'TOTAL STR._attemps_opponent',
       'TD_landed_opponent', 'TD_attemps_opponent', 'HEAD_landed_opponent',
       'HEAD_attemps_opponent', 'BODY_landed_opponent',
       'BODY_attemps_opponent', 'LEG_landed_opponent', 'LEG_attemps_opponent',
       'DISTANCE_landed_opponent', 'DISTANCE_attemps_opponent',
       'CLINCH_landed_opponent', 'CLINCH_attemps_opponent',
       'GROUND_landed_opponent', 'GROUND_attemps_opponent']]

In [105]:
df_total.head()

,EVENT,BOUT,DATE_fighter,WeightClass,Tittle,Women_fight,METHOD,TIME FORMAT,ROUND,TIME,...,BODY_landed_opponent,BODY_attemps_opponent,LEG_landed_opponent,LEG_attemps_opponent,DISTANCE_landed_opponent,DISTANCE_attemps_opponent,CLINCH_landed_opponent,CLINCH_attemps_opponent,GROUND_landed_opponent,GROUND_attemps_opponent
0,UFC 2: No Way Out,Frank Hamaker vs. Thaddeus Luster,1994-03-11,Open Weight,0,0,Submission,[0],1,4:52,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,UFC 2: No Way Out,Frank Hamaker vs. Thaddeus Luster,1994-03-11,Open Weight,0,0,Submission,[0],1,4:52,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,2.0
2,UFC 2: No Way Out,Jason DeLucia vs. Scott Baker,1994-03-11,Open Weight,0,0,Submission,[0],1,6:41,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
3,UFC 2: No Way Out,Jason DeLucia vs. Scott Baker,1994-03-11,Open Weight,0,0,Submission,[0],1,6:41,...,1.0,2.0,1.0,1.0,2.0,4.0,0.0,0.0,1.0,1.0
4,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,1994-03-11,Open Weight,0,0,KO/TKO,[0],1,12:13,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,9.0,15.0


In [106]:
df_total['METHOD'] = df_total['METHOD'].apply(lambda x: 'Decision' if 'Decision' in x else x.strip())
#df_total['METHOD'] = df_total['METHOD'].apply(lambda x: 'KO/TKO' if 'TKO' in x else x.strip())

In [107]:
ground = ['submission', 'guard', 'control', 'mount', 'crucifix']
clinch = ['clinch']
a = 'Punches to Head From GuardSubmission to Strikes'

In [108]:
np.sum([1 if x in a.lower() else 0 for x in ground])

2

In [109]:
def split_TKO(detail):
  lower = detail.lower()
  ground = ['submission', 'guard', 'control', 'mount', 'crucifix', 'ground']
  distance = ['kick', 'distance', 'fist', 'knee', 'elbow']
  clinch = ['clinch']
  injury = ['injury']
  if np.sum([1 if x in lower else 0 for x in ground]) > 0:
    return 'KO/TKO - ground'    
  elif np.sum([1 if x in lower else 0 for x in clinch]) > 0:
    return 'KO/TKO - clinch'
  elif np.sum([1 if x in lower else 0 for x in distance]) > 0:
    return 'KO/TKO - distance'
  elif np.sum([1 if x in lower else 0 for x in injury]) > 0:
    return 'KO/TKO - injury'
  else:
    return 'KO/TKO - other'

In [110]:
split_TKO('Punches to Head From GuardSubmission to Strikes')

'KO/TKO - ground'

In [111]:
df_total.columns

Index(['EVENT', 'BOUT', 'DATE_fighter', 'WeightClass', 'Tittle', 'Women_fight',
       'METHOD', 'TIME FORMAT', 'ROUND', 'TIME', 'TotalTime', 'REFEREE',
       'DETAILS', 'URL', 'FIGHTER', 'Win', 'DRAW', 'No_contest',
       'HEIGHT_fighter', 'WEIGHT_fighter', 'REACH_fighter', 'STANCE_fighter',
       'AGE_fighter', 'fightNo_fighter', 'OPPONENT', 'HEIGHT_opponent',
       'WEIGHT_opponent', 'REACH_opponent', 'STANCE_opponent', 'AGE_opponent',
       'fightNo_opponent', 'KD_fighter', 'SUB.ATT_fighter', 'REV._fighter',
       'CTRL_fighter', 'SIG.STR._landed_fighter', 'SIG.STR._attemps_fighter',
       'TOTAL STR._landed_fighter', 'TOTAL STR._attemps_fighter',
       'TD_landed_fighter', 'TD_attemps_fighter', 'HEAD_landed_fighter',
       'HEAD_attemps_fighter', 'BODY_landed_fighter', 'BODY_attemps_fighter',
       'LEG_landed_fighter', 'LEG_attemps_fighter', 'DISTANCE_landed_fighter',
       'DISTANCE_attemps_fighter', 'CLINCH_landed_fighter',
       'CLINCH_attemps_fighter', 'GROUND_la

In [112]:
df_total['METHOD'].unique()

array(['Submission', 'KO/TKO', "TKO - Doctor's Stoppage", 'Other',
       'Decision', 'Overturned', 'DQ', 'Could Not Continue'], dtype=object)

In [113]:
df_total['DETAILS']

0                                 Keylock From Half Guard 
1                                 Keylock From Half Guard 
2                               Triangle Choke From Mount 
3                               Triangle Choke From Mount 
4          Punches to Head From GuardSubmission to Strikes
                               ...                        
13779    Mike Bell 27 - 30.Derek Cleary 27 - 30.Chris L...
13780                          Guillotine Choke On Ground 
13781                          Guillotine Choke On Ground 
13782    Mike Bell 27 - 30.Adalaide Byrd 28 - 29.Ron Mc...
13783    Mike Bell 27 - 30.Adalaide Byrd 28 - 29.Ron Mc...
Name: DETAILS, Length: 13784, dtype: object

In [114]:
df_total['METHOD'] = df_total.apply(lambda row: split_TKO(row['DETAILS']) if row['METHOD'] =='KO/TKO' else row['METHOD'], axis=1)

In [115]:
df_total[df_total['METHOD']=='KO/TKO - other']['DETAILS'].unique()

array(['toCorner Stoppage', 'toFatigue',
       'toReferee stoppage between rounds', 'Punch to Head From Bottom ',
       'to ', 'toReferee Stoppage Between Rounds',
       'toStoppage Between Rounds'], dtype=object)

In [116]:
df_total.loc[df_total['METHOD'].str.contains('KO'),['METHOD', 'DETAILS']].tail(50)

,METHOD,DETAILS
13674,KO/TKO - distance,Punches to Head At Distance
13675,KO/TKO - distance,Punches to Head At Distance
13680,KO/TKO - distance,Punch to Head At Distance
13681,KO/TKO - distance,Punch to Head At Distance
13682,KO/TKO - distance,Punch to Head At Distance
13683,KO/TKO - distance,Punch to Head At Distance
13684,KO/TKO - distance,Kick to Head At DistanceWheel Kick
13685,KO/TKO - distance,Kick to Head At DistanceWheel Kick
13692,KO/TKO - distance,Punches to Head At Distance
13693,KO/TKO - distance,Punches to Head At Distance


In [117]:
df_total = df_total.rename(columns={"DATE_fighter": "DATE_Event"})

In [118]:
df_total['REACH_fighter'].unique()

array([nan, 72., 75., 74., 76., 70., 73., 77., 68., 78., 67., 79., 69.,
       80., 71., 64., 66., 82., 81., 84., 63., 65., 62., 60., 83., 61.,
       58., 59.])

In [119]:
df_total.iloc[:-5,18:23]

,HEIGHT_fighter,WEIGHT_fighter,REACH_fighter,STANCE_fighter,AGE_fighter
0,NaN,NaN,NaN,NaN,NaN
1,75.0,210.0,NaN,NaN,NaN
2,71.0,190.0,NaN,Southpaw,24.0
3,72.0,210.0,NaN,NaN,NaN
4,77.0,275.0,NaN,NaN,NaN
...,...,...,...,...,...
13774,71.0,170.0,73.0,Orthodox,28.0
13775,73.0,170.0,75.0,Switch,30.0
13776,71.0,185.0,77.0,Switch,38.0
13777,73.0,185.0,76.0,Orthodox,31.0


In [120]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13784 entries, 0 to 13783
Data columns (total 75 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   EVENT                        13784 non-null  object        
 1   BOUT                         13784 non-null  object        
 2   DATE_Event                   13784 non-null  datetime64[ns]
 3   WeightClass                  13784 non-null  object        
 4   Tittle                       13784 non-null  int64         
 5   Women_fight                  13784 non-null  int64         
 6   METHOD                       13784 non-null  object        
 7   TIME FORMAT                  13784 non-null  object        
 8   ROUND                        13784 non-null  int64         
 9   TIME                         13784 non-null  object        
 10  TotalTime                    13784 non-null  int64         
 11  REFEREE                      13720 non-nu

In [121]:
df_total.columns

Index(['EVENT', 'BOUT', 'DATE_Event', 'WeightClass', 'Tittle', 'Women_fight',
       'METHOD', 'TIME FORMAT', 'ROUND', 'TIME', 'TotalTime', 'REFEREE',
       'DETAILS', 'URL', 'FIGHTER', 'Win', 'DRAW', 'No_contest',
       'HEIGHT_fighter', 'WEIGHT_fighter', 'REACH_fighter', 'STANCE_fighter',
       'AGE_fighter', 'fightNo_fighter', 'OPPONENT', 'HEIGHT_opponent',
       'WEIGHT_opponent', 'REACH_opponent', 'STANCE_opponent', 'AGE_opponent',
       'fightNo_opponent', 'KD_fighter', 'SUB.ATT_fighter', 'REV._fighter',
       'CTRL_fighter', 'SIG.STR._landed_fighter', 'SIG.STR._attemps_fighter',
       'TOTAL STR._landed_fighter', 'TOTAL STR._attemps_fighter',
       'TD_landed_fighter', 'TD_attemps_fighter', 'HEAD_landed_fighter',
       'HEAD_attemps_fighter', 'BODY_landed_fighter', 'BODY_attemps_fighter',
       'LEG_landed_fighter', 'LEG_attemps_fighter', 'DISTANCE_landed_fighter',
       'DISTANCE_attemps_fighter', 'CLINCH_landed_fighter',
       'CLINCH_attemps_fighter', 'GROUND_land

In [122]:
cols_to_sum = ['Win', 'DRAW', 'No_contest', 'TotalTime','KD_fighter', 'SUB.ATT_fighter', 'REV._fighter',
       'CTRL_fighter', 'SIG.STR._landed_fighter', 'SIG.STR._attemps_fighter',
       'TOTAL STR._landed_fighter', 'TOTAL STR._attemps_fighter',
       'TD_landed_fighter', 'TD_attemps_fighter', 'HEAD_landed_fighter',
       'HEAD_attemps_fighter', 'BODY_landed_fighter', 'BODY_attemps_fighter',
       'LEG_landed_fighter', 'LEG_attemps_fighter', 'DISTANCE_landed_fighter',
       'DISTANCE_attemps_fighter', 'CLINCH_landed_fighter',
       'CLINCH_attemps_fighter', 'GROUND_landed_fighter',
       'GROUND_attemps_fighter', 'KD_opponent', 'SUB.ATT_opponent',
       'REV._opponent', 'CTRL_opponent', 'SIG.STR._landed_opponent',
       'SIG.STR._attemps_opponent', 'TOTAL STR._landed_opponent',
       'TOTAL STR._attemps_opponent', 'TD_landed_opponent',
       'TD_attemps_opponent', 'HEAD_landed_opponent', 'HEAD_attemps_opponent',
       'BODY_landed_opponent', 'BODY_attemps_opponent', 'LEG_landed_opponent',
       'LEG_attemps_opponent', 'DISTANCE_landed_opponent',
       'DISTANCE_attemps_opponent', 'CLINCH_landed_opponent',
       'CLINCH_attemps_opponent', 'GROUND_landed_opponent',
       'GROUND_attemps_opponent']

In [123]:
for col in list(df_total.columns):
  if '_fighter' in col:
    col_prefix = col.replace('_fighter','' )
    if not col == 'STANCE_fighter':
      print(col, col_prefix)
      df_total.loc[:,col_prefix+'_diff'] = df_total[col].astype(float) - df_total[col_prefix +'_opponent'].astype(float)

HEIGHT_fighter HEIGHT
WEIGHT_fighter WEIGHT
REACH_fighter REACH
AGE_fighter AGE
fightNo_fighter fightNo
KD_fighter KD
SUB.ATT_fighter SUB.ATT
REV._fighter REV.
CTRL_fighter CTRL
SIG.STR._landed_fighter SIG.STR._landed
SIG.STR._attemps_fighter SIG.STR._attemps
TOTAL STR._landed_fighter TOTAL STR._landed
TOTAL STR._attemps_fighter TOTAL STR._attemps
TD_landed_fighter TD_landed
TD_attemps_fighter TD_attemps
HEAD_landed_fighter HEAD_landed
HEAD_attemps_fighter HEAD_attemps
BODY_landed_fighter BODY_landed
BODY_attemps_fighter BODY_attemps
LEG_landed_fighter LEG_landed
LEG_attemps_fighter LEG_attemps
DISTANCE_landed_fighter DISTANCE_landed
DISTANCE_attemps_fighter DISTANCE_attemps
CLINCH_landed_fighter CLINCH_landed
CLINCH_attemps_fighter CLINCH_attemps
GROUND_landed_fighter GROUND_landed
GROUND_attemps_fighter GROUND_attemps


In [124]:
def forma(fighter_name, datum):
  vysledek=''
  skore = 0
  koef =0.5
  result = ['W' if x==1 else 'L' for x in df_total['Win'][(df_total['FIGHTER']==fighter_name) & (df_total['DATE_Event']<datum)]]
  for vyhra in result[:-6:-1]:
    if vyhra =='W':
      skore+=koef
    else:
      skore-=koef
    koef-=0.1  
    vysledek +=vyhra+' '
  vysledek=vysledek[:-1]  
  return (vysledek,skore)

In [125]:
df_total.loc[:,"form_skore_fighter"] = df_total.apply(lambda x:forma(x.FIGHTER, x.DATE_Event)[1], axis=1).astype(float)
df_total.loc[:,"form_skore_opponent"] = df_total.apply(lambda x:forma(x.OPPONENT, x.DATE_Event)[1], axis=1).astype(float)

In [126]:
df_total.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13784 entries, 0 to 13783
Data columns (total 104 columns):
 #    Column                       Non-Null Count  Dtype         
---   ------                       --------------  -----         
 0    EVENT                        13784 non-null  object        
 1    BOUT                         13784 non-null  object        
 2    DATE_Event                   13784 non-null  datetime64[ns]
 3    WeightClass                  13784 non-null  object        
 4    Tittle                       13784 non-null  int64         
 5    Women_fight                  13784 non-null  int64         
 6    METHOD                       13784 non-null  object        
 7    TIME FORMAT                  13784 non-null  object        
 8    ROUND                        13784 non-null  int64         
 9    TIME                         13784 non-null  object        
 10   TotalTime                    13784 non-null  int64         
 11   REFEREE                   

In [127]:
df_total[['FIGHTER', 'form_skore_fighter']].tail(10)

,FIGHTER,form_skore_fighter
13774,Jake Matthews,0.7
13775,Matthew Semelsberger,0.1
13776,Jared Cannonier,0.1
13777,Sean Strickland,0.5
13778,Maheshate,0.5
13779,Rafa Garcia,-0.1
13780,Said Nurmagomedov,1.1
13781,Saidyokub Kakhramonov,0.9
13782,Journey Newson,-0.4
13783,Sergey Morozov,0.2


In [128]:
dummies = pd.get_dummies(df_total['METHOD'])
dummies['Other'] = dummies['Other'] + dummies['Could Not Continue'] + dummies['DQ'] + dummies['KO/TKO - injury'] + dummies['KO/TKO - other'] + dummies['Overturned'] + dummies['TKO - Doctor\'s Stoppage']
dummies = dummies.drop(['Could Not Continue', 'DQ', 'KO/TKO - injury', 'KO/TKO - other', 'Overturned','TKO - Doctor\'s Stoppage' ], axis = 1)
dummies

,Decision,KO/TKO - clinch,KO/TKO - distance,KO/TKO - ground,Other,Submission
0,0,0,0,0,0,1
1,0,0,0,0,0,1
2,0,0,0,0,0,1
3,0,0,0,0,0,1
4,0,0,0,1,0,0
...,...,...,...,...,...,...
13779,1,0,0,0,0,0
13780,0,0,0,0,0,1
13781,0,0,0,0,0,1
13782,1,0,0,0,0,0


In [129]:
fighters_total_df = df_total.pivot_table(index=['FIGHTER', 'HEIGHT_fighter', 'WEIGHT_fighter', 'REACH_fighter', 'STANCE_fighter'], values=cols_to_sum, aggfunc=np.sum)[cols_to_sum].reset_index()

In [130]:
fighters_total_df['Fights'] = fighters_total_df['FIGHTER'].apply(lambda x: df_total['FIGHTER'].tolist().count(x))

In [131]:
fighters_total_df['Lost'] = fighters_total_df['Fights'] - fighters_total_df['Win'] - fighters_total_df['DRAW'] - fighters_total_df['No_contest']

In [132]:
fighters_total_df = fighters_total_df[['FIGHTER', 'HEIGHT_fighter', 'WEIGHT_fighter', 'REACH_fighter',
       'STANCE_fighter', 'Fights', 'Win', 'DRAW', 'Lost','No_contest' , 'TotalTime',
       'KD_fighter', 'SUB.ATT_fighter', 'REV._fighter', 'CTRL_fighter',
       'SIG.STR._landed_fighter', 'SIG.STR._attemps_fighter',
       'TOTAL STR._landed_fighter', 'TOTAL STR._attemps_fighter',
       'TD_landed_fighter', 'TD_attemps_fighter', 'HEAD_landed_fighter',
       'HEAD_attemps_fighter', 'BODY_landed_fighter', 'BODY_attemps_fighter',
       'LEG_landed_fighter', 'LEG_attemps_fighter', 'DISTANCE_landed_fighter',
       'DISTANCE_attemps_fighter', 'CLINCH_landed_fighter',
       'CLINCH_attemps_fighter', 'GROUND_landed_fighter',
       'GROUND_attemps_fighter', 'KD_opponent', 'SUB.ATT_opponent',
       'REV._opponent', 'CTRL_opponent', 'SIG.STR._landed_opponent',
       'SIG.STR._attemps_opponent', 'TOTAL STR._landed_opponent',
       'TOTAL STR._attemps_opponent', 'TD_landed_opponent',
       'TD_attemps_opponent', 'HEAD_landed_opponent', 'HEAD_attemps_opponent',
       'BODY_landed_opponent', 'BODY_attemps_opponent', 'LEG_landed_opponent',
       'LEG_attemps_opponent', 'DISTANCE_landed_opponent',
       'DISTANCE_attemps_opponent', 'CLINCH_landed_opponent',
       'CLINCH_attemps_opponent', 'GROUND_landed_opponent',
       'GROUND_attemps_opponent']]

In [133]:
fighters_total_df.sort_values('Fights', ascending=False)

,FIGHTER,HEIGHT_fighter,WEIGHT_fighter,REACH_fighter,STANCE_fighter,Fights,Win,DRAW,Lost,No_contest,...,BODY_landed_opponent,BODY_attemps_opponent,LEG_landed_opponent,LEG_attemps_opponent,DISTANCE_landed_opponent,DISTANCE_attemps_opponent,CLINCH_landed_opponent,CLINCH_attemps_opponent,GROUND_landed_opponent,GROUND_attemps_opponent
726,Jim Miller,68.0,155.0,71.0,Southpaw,40,24,0,15,1,...,244.0,343.0,111.0,148.0,799.0,2238.0,162.0,241.0,174.0,281.0
93,Andrei Arlovski,75.0,240.0,77.0,Orthodox,39,23,0,15,1,...,204.0,297.0,223.0,256.0,904.0,2329.0,156.0,242.0,158.0,235.0
438,Donald Cerrone,73.0,170.0,73.0,Orthodox,38,23,0,14,1,...,444.0,668.0,312.0,379.0,1558.0,3375.0,211.0,305.0,56.0,88.0
708,Jeremy Stephens,69.0,145.0,71.0,Orthodox,34,15,0,18,1,...,210.0,307.0,261.0,316.0,939.0,2448.0,108.0,158.0,142.0,228.0
300,Clay Guida,67.0,155.0,70.0,Orthodox,34,18,0,16,0,...,195.0,277.0,92.0,115.0,847.0,2643.0,189.0,263.0,114.0,138.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750,Joe Pyfer,74.0,185.0,75.0,Orthodox,1,1,0,0,0,...,2.0,4.0,5.0,7.0,11.0,30.0,0.0,0.0,0.0,0.0
747,Joe Giannetti,72.0,155.0,74.0,Southpaw,1,0,0,1,0,...,7.0,18.0,26.0,27.0,40.0,67.0,0.0,0.0,2.0,4.0
730,Jimmy Flick,67.0,125.0,68.0,Orthodox,1,1,0,0,0,...,3.0,3.0,1.0,1.0,17.0,28.0,2.0,2.0,2.0,2.0
692,Javier Vazquez,67.0,145.0,69.0,Southpaw,1,1,0,0,0,...,6.0,10.0,3.0,4.0,27.0,74.0,0.0,2.0,0.0,0.0


In [134]:
df_total['METHOD'].unique()

array(['Submission', 'KO/TKO - ground', 'KO/TKO - other',
       'KO/TKO - clinch', "TKO - Doctor's Stoppage", 'Other',
       'KO/TKO - distance', 'Decision', 'Overturned', 'DQ',
       'KO/TKO - injury', 'Could Not Continue'], dtype=object)

In [135]:
fighters_total_df.columns

Index(['FIGHTER', 'HEIGHT_fighter', 'WEIGHT_fighter', 'REACH_fighter',
       'STANCE_fighter', 'Fights', 'Win', 'DRAW', 'Lost', 'No_contest',
       'TotalTime', 'KD_fighter', 'SUB.ATT_fighter', 'REV._fighter',
       'CTRL_fighter', 'SIG.STR._landed_fighter', 'SIG.STR._attemps_fighter',
       'TOTAL STR._landed_fighter', 'TOTAL STR._attemps_fighter',
       'TD_landed_fighter', 'TD_attemps_fighter', 'HEAD_landed_fighter',
       'HEAD_attemps_fighter', 'BODY_landed_fighter', 'BODY_attemps_fighter',
       'LEG_landed_fighter', 'LEG_attemps_fighter', 'DISTANCE_landed_fighter',
       'DISTANCE_attemps_fighter', 'CLINCH_landed_fighter',
       'CLINCH_attemps_fighter', 'GROUND_landed_fighter',
       'GROUND_attemps_fighter', 'KD_opponent', 'SUB.ATT_opponent',
       'REV._opponent', 'CTRL_opponent', 'SIG.STR._landed_opponent',
       'SIG.STR._attemps_opponent', 'TOTAL STR._landed_opponent',
       'TOTAL STR._attemps_opponent', 'TD_landed_opponent',
       'TD_attemps_opponent', 'HE

# **df_pro_model**

In [136]:
cols_to_sum = ['Win', 'DRAW', 'No_contest', 'TotalTime','KD_fighter', 'SUB.ATT_fighter', 'REV._fighter',
       'CTRL_fighter', 'SIG.STR._landed_fighter', 'SIG.STR._attemps_fighter',
       'TOTAL STR._landed_fighter', 'TOTAL STR._attemps_fighter',
       'TD_landed_fighter', 'TD_attemps_fighter', 'HEAD_landed_fighter',
       'HEAD_attemps_fighter', 'BODY_landed_fighter', 'BODY_attemps_fighter',
       'LEG_landed_fighter', 'LEG_attemps_fighter', 'DISTANCE_landed_fighter',
       'DISTANCE_attemps_fighter', 'CLINCH_landed_fighter',
       'CLINCH_attemps_fighter', 'GROUND_landed_fighter',
       'GROUND_attemps_fighter', 'KD_opponent', 'SUB.ATT_opponent',
       'REV._opponent', 'CTRL_opponent', 'SIG.STR._landed_opponent',
       'SIG.STR._attemps_opponent', 'TOTAL STR._landed_opponent',
       'TOTAL STR._attemps_opponent', 'TD_landed_opponent',
       'TD_attemps_opponent', 'HEAD_landed_opponent', 'HEAD_attemps_opponent',
       'BODY_landed_opponent', 'BODY_attemps_opponent', 'LEG_landed_opponent',
       'LEG_attemps_opponent', 'DISTANCE_landed_opponent',
       'DISTANCE_attemps_opponent', 'CLINCH_landed_opponent',
       'CLINCH_attemps_opponent', 'GROUND_landed_opponent',
       'GROUND_attemps_opponent']

In [137]:
df_total.groupby('FIGHTER')[cols_to_sum].transform(lambda x: x.shift(1).cumsum())

,Win,DRAW,No_contest,TotalTime,KD_fighter,SUB.ATT_fighter,REV._fighter,CTRL_fighter,SIG.STR._landed_fighter,SIG.STR._attemps_fighter,...,BODY_landed_opponent,BODY_attemps_opponent,LEG_landed_opponent,LEG_attemps_opponent,DISTANCE_landed_opponent,DISTANCE_attemps_opponent,CLINCH_landed_opponent,CLINCH_attemps_opponent,GROUND_landed_opponent,GROUND_attemps_opponent
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13779,2.0,0.0,0.0,4190.0,0.0,2.0,0.0,1195.0,228.0,591.0,...,121.0,139.0,33.0,37.0,277.0,644.0,50.0,59.0,5.0,7.0
13780,5.0,0.0,0.0,2946.0,1.0,1.0,0.0,168.0,204.0,443.0,...,24.0,51.0,43.0,58.0,91.0,253.0,9.0,14.0,8.0,13.0
13781,2.0,0.0,0.0,1779.0,0.0,4.0,0.0,806.0,68.0,124.0,...,9.0,16.0,4.0,4.0,19.0,53.0,6.0,6.0,2.0,5.0
13782,1.0,0.0,1.0,1879.0,1.0,1.0,0.0,57.0,102.0,281.0,...,15.0,30.0,16.0,20.0,76.0,224.0,2.0,5.0,4.0,4.0


In [138]:
df_pre_comp = df_total[['BOUT', 'FIGHTER','METHOD','Win','Women_fight','ROUND','HEIGHT_diff','REACH_diff','AGE_diff','form_skore_fighter', 'form_skore_opponent','fightNo_fighter','fightNo_opponent']]

In [139]:
df_pre_comp = df_pre_comp.join(df_total.groupby('FIGHTER')[cols_to_sum].transform(lambda x: x.shift(1).cumsum()),rsuffix='_tot')

In [140]:
df_pre_comp.tail()

,BOUT,FIGHTER,METHOD,Win,Women_fight,ROUND,HEIGHT_diff,REACH_diff,AGE_diff,form_skore_fighter,...,BODY_landed_opponent,BODY_attemps_opponent,LEG_landed_opponent,LEG_attemps_opponent,DISTANCE_landed_opponent,DISTANCE_attemps_opponent,CLINCH_landed_opponent,CLINCH_attemps_opponent,GROUND_landed_opponent,GROUND_attemps_opponent
13779,Rafa Garcia vs. Maheshate,Rafa Garcia,Decision,1,0,3,-5.0,-1.0,5.0,-0.1,...,121.0,139.0,33.0,37.0,277.0,644.0,50.0,59.0,5.0,7.0
13780,Said Nurmagomedov vs. Saidyokub Kakhramonov,Said Nurmagomedov,Submission,1,0,2,0.0,1.0,3.0,1.1,...,24.0,51.0,43.0,58.0,91.0,253.0,9.0,14.0,8.0,13.0
13781,Said Nurmagomedov vs. Saidyokub Kakhramonov,Saidyokub Kakhramonov,Submission,0,0,2,0.0,-1.0,-3.0,0.9,...,9.0,16.0,4.0,4.0,19.0,53.0,6.0,6.0,2.0,5.0
13782,Sergey Morozov vs. Journey Newson,Journey Newson,Decision,0,0,3,-1.0,0.0,0.0,-0.4,...,15.0,30.0,16.0,20.0,76.0,224.0,2.0,5.0,4.0,4.0
13783,Sergey Morozov vs. Journey Newson,Sergey Morozov,Decision,1,0,3,1.0,0.0,0.0,0.2,...,38.0,68.0,30.0,35.0,141.0,347.0,16.0,19.0,1.0,4.0


In [141]:
df_pre_comp[df_pre_comp['FIGHTER']=='Conor McGregor']

,BOUT,FIGHTER,METHOD,Win,Women_fight,ROUND,HEIGHT_diff,REACH_diff,AGE_diff,form_skore_fighter,...,BODY_landed_opponent,BODY_attemps_opponent,LEG_landed_opponent,LEG_attemps_opponent,DISTANCE_landed_opponent,DISTANCE_attemps_opponent,CLINCH_landed_opponent,CLINCH_attemps_opponent,GROUND_landed_opponent,GROUND_attemps_opponent
4448,Marcus Brimage vs. Conor McGregor,Conor McGregor,KO/TKO - distance,1,0,1,5.0,3.0,-4.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4704,Conor McGregor vs. Max Holloway,Conor McGregor,Decision,1,0,3,-2.0,5.0,4.0,0.5,...,2.0,2.0,3.0,5.0,12.0,41.0,0.0,0.0,0.0,0.0
5604,Conor McGregor vs. Diego Brandao,Conor McGregor,KO/TKO - distance,1,0,1,2.0,10.0,-1.0,0.9,...,3.0,6.0,15.0,23.0,34.0,133.0,1.0,1.0,0.0,0.0
5792,Dustin Poirier vs. Conor McGregor,Conor McGregor,KO/TKO - distance,1,0,1,0.0,2.0,1.0,1.2,...,7.0,11.0,15.0,24.0,39.0,149.0,5.0,6.0,0.0,1.0
6080,Conor McGregor vs. Dennis Siver,Conor McGregor,KO/TKO - distance,1,0,2,3.0,4.0,-10.0,1.4,...,7.0,11.0,22.0,32.0,49.0,168.0,5.0,6.0,0.0,1.0
6533,Chad Mendes vs. Conor McGregor,Conor McGregor,KO/TKO - distance,1,0,2,3.0,8.0,-4.0,1.5,...,11.0,24.0,37.0,55.0,79.0,266.0,7.0,12.0,0.0,1.0
6962,Jose Aldo vs. Conor McGregor,Conor McGregor,KO/TKO - distance,1,0,1,2.0,4.0,-2.0,1.5,...,12.0,25.0,39.0,57.0,93.0,291.0,8.0,15.0,16.0,21.0
7156,Conor McGregor vs. Nate Diaz,Conor McGregor,Submission,0,0,2,-3.0,-2.0,-3.0,1.5,...,12.0,25.0,39.0,57.0,94.0,294.0,8.0,15.0,16.0,21.0
7574,Nate Diaz vs. Conor McGregor,Conor McGregor,Decision,1,0,5,-3.0,-2.0,-3.0,0.5,...,21.0,38.0,42.0,65.0,156.0,427.0,18.0,27.0,21.0,28.0
7772,Eddie Alvarez vs. Conor McGregor,Conor McGregor,KO/TKO - distance,1,0,2,0.0,5.0,-4.0,0.7,...,71.0,109.0,51.0,81.0,244.0,654.0,95.0,139.0,22.0,32.0


In [142]:
df_total['FIGHTER'].tail()

13779              Rafa Garcia
13780        Said Nurmagomedov
13781    Saidyokub Kakhramonov
13782           Journey Newson
13783           Sergey Morozov
Name: FIGHTER, dtype: object

In [143]:
df_fighter_tott.to_csv('fighter_details.csv')
fighters_total_df.to_csv('fighter_total_stats.csv')
df_total.to_csv('fight_with_stats.csv')
df_pre_comp.to_csv('fight_with_stats_precomp.csv')
df_results.to_csv('fight_result.csv')